# Visualize superpoint and deep fundamental matrix
- based on deep_F_baselineEval

In [ ]:
# import sys
# sys.path.append(
#     "/home/yyjau/Documents/deepSfm/torch-batch-svd"
# )  # path for youyi in theia
# %reload_ext autoreload
# %autoreload 2

In [ ]:
# import os
# os.environ['CUDA_DEVICE_ORDER']=“PCI_BUS_ID”
# os.environ[“CUDA_VISIBLE_DEVICES”]=“5”

In [ ]:
%reload_ext autoreload
%autoreload 2

import argparse
import yaml
import os, sys
from path import Path
import copy
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import cv2
import time
import scipy
from tqdm import tqdm
from utils.logging import *
import argparse
from pebble import ProcessPool
import multiprocessing as mp
ratio_CPU = 0.5
default_number_of_process = int(ratio_CPU * mp.cpu_count())

import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim

from tensorboardX import SummaryWriter
from settings import EXPER_PATH


## loaders: data, model, pretrained model
from utils.loader import dataLoader, modelLoader, pretrainedLoader_net, pretrainedLoader_opt
sys.path.append('/home/ruizhu/Documents/Projects/kitti_instance_RGBD_utils')
import dsac_tools.utils_F as utils_F # If cannot find: export KITTI_UTILS_PATH='/home/ruizhu/Documents/Projects/kitti_instance_RGBD_utils'
import dsac_tools.utils_opencv as utils_opencv # If cannot find: export KITTI_UTILS_PATH='/home/ruizhu/Documents/Projects/kitti_instance_RGBD_utils'
import dsac_tools.utils_vis as utils_vis # If cannot find: export KITTI_UTILS_PATH='/home/ruizhu/Documents/Projects/kitti_instance_RGBD_utils'
import dsac_tools.utils_misc as utils_misc # If cannot find: export KITTI_UTILS_PATH='/home/ruizhu/Documents/Projects/kitti_instance_RGBD_utils'
import dsac_tools.utils_geo as utils_geo # If cannot find: export KITTI_UTILS_PATH='/home/ruizhu/Documents/Projects/kitti_instance_RGBD_utils'

from train_good_utils import get_all_loss, val_rt, get_all_loss_DeepF

# from utils.utils import tensor2array, save_checkpoint, load_checkpoint, save_path_formatter
# from utils.utils import getWriterPath
# from utils.utils import saveLoss

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

torch.set_default_tensor_type(torch.FloatTensor)
logging.basicConfig(format='[%(asctime)s %(levelname)s] %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S', level=logging.DEBUG)

# add parser
parser = argparse.ArgumentParser(description='Foo')

# Training command
parser.add_argument('config', type=str)
parser.add_argument('exper_name', type=str)
# args = parser.parse_args('configs/apollo_corr_baseline.yaml baselineEval_visKitti'.split())
# args = parser.parse_args('configs/apollo_train_corr_baseline.yaml baselineEval_visKitti'.split())
args = parser.parse_args('configs/apollo_train_corr_baselineEval.yaml baselineEval_visKitti'.split())
# args = parser.parse_args('configs/kitti_corr_baselineEval.yaml baselineEval_visKitti'.split())
# args = parser.parse_args('configs/euroc_corr_baselineEval.yaml baselineEval_visKitti'.split())
print(args)

with open(args.config, 'r') as f:
    config = yaml.load(f)
# EXPER_PATH from settings.py
output_dir = os.path.join(EXPER_PATH, args.exper_name)
print(output_dir)
print(f"config: {config}")

In [ ]:
config['data']['batch_size'] = 1
# config['data']['read_what']['with_imgs'] = False
# config['training']['pretrained']

In [ ]:
# # config['training']['pretrained'] = 'logs/mm1-testCondition_lossRes_regCondClip1e+4Balance1e-6/checkpoints/superPointNet_96000_checkpoint.pth.tar'
# # config['training']['pretrained'] = 'logs/mm1-retro_clamp02_normHW/checkpoints/superPointNet_111500_checkpoint.pth.tar'
# # config['training']['pretrained'] = 'logs/mm2-retro_clamp02_normHW_offset/checkpoints/superPointNet_45500_checkpoint.pth.tar'
# # config['training']['pretrained'] = 'logs/mm1-retro_clamp02_normHW_finetuneQtLoss-resetIter-first1000DeepFLoss_balanceQ10t1_lr1e-4/checkpoints/superPointNet_185000_checkpoint.pth.tar'
# # config['training']['pretrained'] = 'logs/test_pose_from_start_3800+/checkpoints/superPointNet_29800_checkpoint.pth.tar'
# config['training']['pretrained'] = 'logs/baselineTrain_sp_deepF_kitti_qtLoss_v0/checkpoints/deepFNet_30000_checkpoint.pth.tar'

# # edit data root
# config['data']['dump_root'] = '/home/ruizhu/Documents/Datasets/kitti/kitti_dump/odo_corr_dump_siftIdx_npy_delta1235810_full'

# # config['training']['pretrained'] = 'logs/mm2-retro_clamp02_noSVDNormalize/checkpoints/superPointNet_87500_checkpoint.pth.tar'

# config['data']['batch_size'] = 1
# # config['data']['dump_root'] = '/home/ruizhu/Documents/Datasets/kitti/kitti_dump/odo_corr_dump_siftIdx_npy_delta1235810_full_deepFSplitRe'
# config['data']['delta_ij'] = 1

# config['data']['read_what']['with_des'] = False
# config['data']['read_what']['with_quality'] = False # True # True
# # config['data']['read_what']['with_sift_des'] = True
# config['model']['if_img_feat'] = False
# config['model']['if_quality'] = False
# config['model']['if_learn_offsets'] = False
# config['model']['if_qt_loss'] = False
# config['data']['read_what']['with_X'] = True
# config['model']['if_tri_depth'] = False
# config['model']['if_sample_loss'] = False

# config['training']['reproduce'] = False
# # config['model']['name'] = 'GoodCorresNet_layers_deepF_singleSample'

### Load dataset (only need to do once)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info('train on device: %s', device)
args.test = True
# writer = SummaryWriter(getWriterPath(task='train_good', exper_name=args.exper_name, date=True))

# data loading
assert config['data']['sequence_length']==2, "Sorry dude, we are only supporting two-frame setting for now."
assert (config['data']['read_what']['with_X'] and config['data']['batch_size']==1) or (not config['data']['read_what']['with_X']), 'We are not suppoting batching lidar Xs with batch_size>1 yet!'
val = 'test' if args.test else 'val' 
data = dataLoader(config, dataset=config['data']['dataset'], val=val, warp_input=True, val_shuffle=False)
train_loader, val_loader = data['train_loader'], data['val_loader']
logging.info('+++[Dataset]+++ train split size %d in %d batches, val split size %d in %d batches'%\
    (len(train_loader)*config['data']['batch_size'], len(train_loader), len(val_loader)*config['data']['batch_size'], len(val_loader)))

In [ ]:
samples = data['val_loader'].dataset.samples
print(f"samples: {len(samples)}")
idx = 3
print(f"samples: {samples[idx]}")
sample = samples[idx]

In [ ]:
import matplotlib.pyplot as plt
for j, sample in enumerate(val_loader):
    image_0 = sample['imgs'][0]
    print(f"gamma: {sample['gamma'][0]}, image: {type(image)}, {image.shape}, {image.max()}")
#     plt.imshow(image.squeeze().numpy()/255)
    image_1 = sample['imgs'][1]
    image_1_gray = cv2.cvtColor(image_1, cv2.COLOR_RGB2GRAY)
#     plt.show()
    image = image_0 + 0
    image[0] += image_1_gray
    plt.imshow(image.squeeze().numpy()/255)
    plt.show()
    if j >= 0: break

In [ ]:
def relative_pose_cam_to_body(relative_scene_pose, Rt_cam2_gt):
    """ transform the camera pose from camera coordinate to body coordinate
    """
    relative_scene_pose = (
        np.linalg.inv(Rt_cam2_gt) 
        @ relative_scene_pose
        @ Rt_cam2_gt
    )
    return relative_scene_pose

M_estW = sample['cam_poses'][0]
M_estW = sample['cam_poses'][0]
Rt_cam2_gt_np = sample['Rt_cam2_gt']

# M_estW_body = relative_pose_cam_to_body(M_estW, Rt_cam2_gt_np)
M_estW_body = M_estW
print(f"M_estW_body: {M_estW_body}")
from numpy.linalg import inv
print(f"M_estW_body inv: {inv(M_estW_body)}")

In [ ]:
# %reload_ext autoreload
# %autoreload 2

# # model loading
# img_zoom_xy = (config['data']['preprocessing']['resize'][1]/config['data']['image']['size'][1], config['data']['preprocessing']['resize'][0]/config['data']['image']['size'][0])

# # model_params = {'depth': config['model']['depth'], 'clamp_at':config['model']['clamp_at'], 'in_channels': 4, 'num_seg_classes': 1, 'with_transform': False, 'with_instance_norm': True}
# model_params = {'depth': config['model']['depth'],  'img_zoom_xy': img_zoom_xy, 'image_size': config['data']['image']['size'], \
#     'quality_size': config['model']['quality_size'], 'if_quality': config['model']['if_quality'], \
#     'if_img_des_to_pointnet': config['model']['if_img_des_to_pointnet'], 'if_goodCorresArch': config['model']['if_goodCorresArch'], 'if_img_feat': config['model']['if_img_feat'], \
#     'if_cpu_svd': config['model']['if_cpu_svd'], 'if_learn_offsets': config['model']['if_learn_offsets'], 'if_sample_loss': config['model']['if_sample_loss']}

# # model_params = {'depth': config['model']['depth'],  'zoom_xy': zoom_xy, 'quality_size': config['model']['quality_size'], \
# #     'if_quality': config['model']['if_quality'], 'if_des': config['model']['if_des'], 'if_goodCorresArch': config['model']['if_goodCorresArch'], 'if_img_feat': config['model']['if_img_feat']}
# net_eval = modelLoader(config['model']['name'], model_params)

# logging.info('+++[Train]+++ setting adam solver')
# n_iter = 0
# n_iter_val = 0
# epoch = 0

# ## load pretrained
# checkpoint_path = config['training']['pretrained']
# print(checkpoint_path)
# checkpoint_mode = '' if checkpoint_path[-3:] == 'pth' else 'full'
# logging.info('Loading net: path: %s, mode: %s'%(checkpoint_path, checkpoint_mode))
# net_eval, n_iter, _ = pretrainedLoader_net(net_eval, n_iter, checkpoint_path, mode=checkpoint_mode, full_path=True)

# logging.info("+++[Train]+++ Let's use %d GPUs!"%torch.cuda.device_count())
# net_eval = net_eval.to(device)
# # net = nn.DataParallel(net)
# optimizer = optim.Adam(net_eval.parameters(), lr=config['training']['learning_rate'])
# logging.info('Loading optimizer: path: %s, mode: %s'%(checkpoint_path, checkpoint_mode))
# optimizer, n_iter, _ = pretrainedLoader_opt(optimizer, n_iter, checkpoint_path, mode=checkpoint_mode, full_path=True)

In [ ]:
# # model loading
from train_good_corr_4_vals_goodF_baseline import prepare_model

img_zoom_xy = (
    config["data"]["preprocessing"]["resize"][1]
    / config["data"]["image"]["size"][1],
    config["data"]["preprocessing"]["resize"][0]
    / config["data"]["image"]["size"][0],
)
model_params = {
    "depth": config["model"]["depth"],
    "img_zoom_xy": img_zoom_xy,
    "image_size": config["data"]["image"]["size"],
    "quality_size": config["model"]["quality_size"],
    "if_quality": config["model"]["if_quality"],
    "if_img_des_to_pointnet": config["model"]["if_img_des_to_pointnet"],
    "if_goodCorresArch": config["model"]["if_goodCorresArch"],
    "if_img_feat": config["model"]["if_img_feat"],
    "if_cpu_svd": config["model"]["if_cpu_svd"],
    "if_learn_offsets": config["model"]["if_learn_offsets"],
    "if_tri_depth": config["model"]["if_tri_depth"],
    "if_sample_loss": config["model"]["if_sample_loss"],
}
net = modelLoader(config["model"]["name"], **model_params)
net, optimizer, n_iter, n_iter_val = prepare_model(
    config, net, device, n_iter=0, n_iter_val=0, net_postfix=''
)
net_eval = net.to(device)

# load sp
if config['model']['if_SP']:
    SP_params = {
        "out_num_points": 2000,
        "patch_size": 5,
        "device": device,
        "nms_dist": 4,
        "conf_thresh": 0.015,
    }
    from models.model_utils import SuperPointNet_process
    from models.model_wrap import PointTracker
    from models.SuperPointNet_gauss2 import *
    SP_processer = SuperPointNet_process(**SP_params)
    SP_tracker = PointTracker(max_length=2, nn_thresh=1.2)
    net_SP = SuperPointNet_gauss2()

    net_SP, optimizer_SP, n_iter_SP, n_iter_val_SP = prepare_model(
        config, net_SP, device, n_iter=0, n_iter_val=0, net_postfix='_SP', train=False
    )
    
    logging.info("+++[Train]+++  training superpoint")

## create subgraph for combinations

In [ ]:


frame_list = [0, 100, 200, 300]
prefix = 'Sp-Df-fp-end-k'
plot_folder = 'plots/'
plot_name = 'corr_all_'
image_name = [f"{plot_folder}{plot_name}{prefix}{i:06}_{(i+1):06}.png" for i in frame_list]
print(image_name)
img_num = 4

image_datas = []
for i, file in enumerate(image_name):
    img = cv2.imread(file)
    image_datas.append(img)
    print(f"read {i}: {file}")
#     plt.imshow(img)
#     plt.show()

f, axarr = plt.subplots(2,2, figsize=(48, 12))
for i in range(img_num):
    axarr[int(i/2),i%2].imshow(image_datas[i])
    axarr[int(i/2),i%2].axis('off')
#     axarr[i/2,i%2].imshow(image_datas[1])
#     axarr[1,0].imshow(image_datas[2])
#     axarr[1,1].imshow(image_datas[3])

plt.show()

In [ ]:
## networks

In [ ]:
val_agent.net_dict

# Eval one sample

### before end-to-end training

In [ ]:
# kitti
config['training']['pretrained'] = 'logs/baselineTrain_deepF_kitti_fLoss_v1/checkpoints/deepFNet_30000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = 'logs/superpoint_kitti_heat2_0/checkpoints/superPointNet_50000_checkpoint.pth.tar'
model_name = 'Sp-Df-fp-k'

In [ ]:
# apollo
config['training']['pretrained'] = 'logs/baselineTrain_deepF_apolloFLoss_v3/checkpoints/deepFNet_60000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = 'logs/superpoint_apollo_v1/checkpoints/superPointNet_40000_checkpoint.pth.tar'


In [ ]:
# kitti end-2-end (best)
config['training']['pretrained'] = 'logs/baselineTrain_kittiSp_deepF_end_kittiFLossPoseLoss_v1_freezeSp/checkpoints/deepFNet_38000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = 'logs/baselineTrain_kittiSp_deepF_end_kittiFLossPoseLoss_v1_freezeSp/checkpoints/superPointNet_38000_checkpoint.pth.tar'
config['model']['if_SP'] = True
model_name = 'Sp-Df-fp-end-k' #

In [ ]:
# apollo end-to-end
config['training']['pretrained'] = 'logs/baselineTrain_apolloSp_apolloDeepF_end_apolloFLossPoseLoss_v1_freezeSp_poseLoss_cont/checkpoints/deepFNet_22000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = 'logs/baselineTrain_apolloSp_apolloDeepF_end_apolloFLossPoseLoss_v1_freezeSp_poseLoss_cont/checkpoints/superPointNet_22000_checkpoint.pth.tar'


In [ ]:
# sift 
config['training']['pretrained'] = 'logs/baselineTrain_sift_deepF_poseLoss_v0/checkpoints/deepFNet_40000_checkpoint.pth.tar'
config['model']['if_SP'] = False
config['training']['pretrained_SP'] = ''
model_name = 'Si-Df-fp-k' #


In [ ]:
# kitti freeze deepF
config['training']['pretrained'] = f'logs/baselineTrain_kittiSp_deepF_kittiFLoss_v0/checkpoints/deepFNet_18000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = f'logs/superpoint_kitti_heat2_0/checkpoints/superPointNet_50000_checkpoint.pth.tar'        
config['model']['if_SP'] = True



In [ ]:
# # config['training']['pretrained'] = 'logs/baselineTrain_deepF_kitti_fLoss_v1/checkpoints/deepFNet_30000_checkpoint.pth.tar'
# config['training']['pretrained'] = 'logs/baselineTrain_DeepF_apolloFLoss_fromKitti_v3/checkpoints/deepFNet_30000_checkpoint.pth.tar'
# config['model']['if_SP'] = True; logging.info(f"not using superpoint!")
# # config['model']['if_SP'] = True; logging.info(f"using superpoint!")

# config['training']['pretrained_SP'] = 'logs/superpoint_spollo_v0/checkpoints/superPointNet_50000_checkpoint.pth.tar'


In [ ]:
# for j, sample in enumerate(val_loader):
#     print(f"sample: {list(sample)}")
#     if j==0: break

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from utils.eval_tools import Val_pipeline_frontend

# plot_epipolar_list=["mask_conf", "mask_epi_dist_gt"]
plot_epipolar_list=["mask_conf"]
plot_corr_list=[""]  # "all"
val_agent = Val_pipeline_frontend(config, device=device)
# val_agent.add_net('deepF', net)
# val_agent.add_net('superpointNet', net_SP)
val_agent.load_net_deepF(name='net_deepF')
if config['model']['if_SP']: val_agent.load_net_SP()
val_agent.net_toeval() 

num_list = []
iter_max = 500 # 1000
# iter_list = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# iter_list = np.arange(27)*100
# iter_list = [171, 1775, 2591, 2748]  ## ours good!
# iter_list = [1466, 1599, 2241]  ## ours bad!
# iter_list = [1119, 1510, 2317]  ## ours rotations!
# iter_list = [1605, 2225, 1518]  ## ours translations!
iter_dict = {
#     'test': [0, 5],
#     'good': [171, 1775, 2591, 2748],  ## ours good!
#     'good': [171, 2573, 2741],  ## ours good!
#     'bad': [2450, 2758, 2464],  ## ours bad!
#     'rot': [1493, 1494, 1495, 1496, 657, 658, 659, 923, 924, 925],  ## ours rotations!
#     'rot': [267],  ## ours rotations!
#     'trans': [1605, 2225, 1518]  ## ours translations!
}
## 2020/02/27 output all the frames
iter_dict = {
    'all_fr': np.arange(iter_max),
}

iter_dict_rev = {}
for i, en in enumerate(iter_dict):
    iters = iter_dict[en]
    for it in iters:
        iter_dict_rev[it] = en
print(f"iter_dict_rev: {iter_dict_rev}")


# iter_list = [200]
from tqdm import tqdm

base_folder = 'plots/vis_paper/'+model_name+"_apo"
print(f"save base_folder: {base_folder}")

for j, sample in tqdm(enumerate(val_loader)):
#     val_agent.eval_one_sample(sample)
#     if j in iter_list:

    if j in iter_dict_rev:
        prefix = f'{model_name}_{iter_dict_rev[j]}_{j}' #  
        print(sample['scene_name'], sample['frame_ids'], sample['img_zoom_xy'])
        logging.info(f"eval {j}, prefix: {prefix}")
        val_agent.run_eval(sample, plot_epipolar_list=plot_epipolar_list, 
                           plot_corr_list=plot_corr_list, prefix=prefix, 
                           save=True, title=False, base_folder=base_folder)


    if j >= iter_max:
        break

In [ ]:
171 in iter_dict_rev

### After end-to-end training

In [ ]:
config['training']['pretrained'] = 'logs/baselineTrain_sp_deepF_kitti_qtLoss_v3/checkpoints/deepFNet_8000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = 'logs/baselineTrain_sp_deepF_kitti_qtLoss_v3/checkpoints/superPointNet_8000_checkpoint.pth.tar'


In [ ]:
config['training']['pretrained'] = 'logs/baselineTrain_apolloSp_apolloDeepF_apolloPoseLoss_v0/checkpoints/deepFNet_9000_checkpoint.pth.tar'
config['training']['pretrained_SP'] = 'logs/baselineTrain_apolloSp_apolloDeepF_apolloPoseLoss_v0/checkpoints/superPointNet_9000_checkpoint.pth.tar'


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from utils.eval_tools import Val_pipeline_frontend

val_agent = Val_pipeline_frontend(config, device=device)
# val_agent.add_net('deepF', net)
# val_agent.add_net('superpointNet', net_SP)
val_agent.load_net_deepF(name='net_deepF')
val_agent.load_net_SP()
val_agent.net_toeval() 

num_list = []
iter_max = 3
for j, sample in enumerate(val_loader):
    print(sample['scene_name'], sample['frame_ids'])
#     val_agent.eval_one_sample(sample)
    
    val_agent.run_eval(sample)

    if j >= iter_max:
        break

In [ ]:
# x1_velo = sample['pts1_velo'].squeeze().numpy()[:, :2]
# x2_velo = sample['pts2_velo'].squeeze().numpy()[:, :2]

# utils_vis.draw_corr(img1_rgb, img2_rgb, x1[mask_sample], x2[mask_sample], linewidth=2., title='Sample of 100 corres.')
# utils_vis.draw_corr(img1_rgb, img2_rgb, x1_velo, x2_velo, linewidth=2., title='Sample of 100 corres.')


## How points move when training

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from utils.eval_tools import Val_pipeline_frontend

def change_config(config, iter=1000, mode=0): # 8000 the best
    if mode == 0:
        config['training']['pretrained'] = f'logs/baselineTrain_sp_deepF_kitti_qtLoss_v3/checkpoints/deepFNet_{iter}_checkpoint.pth.tar'
        config['training']['pretrained_SP'] = f'logs/baselineTrain_sp_deepF_kitti_qtLoss_v3/checkpoints/superPointNet_{iter}_checkpoint.pth.tar'
    elif mode == 1: # only change the superpointnet
        config['training']['pretrained'] = 'logs/baselineTrain_deepF_apolloFLoss_v3/checkpoints/deepFNet_60000_checkpoint.pth.tar'
        config['training']['pretrained_SP'] = f'logs/superpoint_apollo_v1/checkpoints/superPointNet_{iter}_checkpoint.pth.tar'
    elif mode == 2:
        config['training']['pretrained'] = 'logs/baselineTrain_deepF_kitti_fLoss_v1/checkpoints/deepFNet_30000_checkpoint.pth.tar'
        config['training']['pretrained_SP'] = f'logs/superpoint_kitti_heat2_0/checkpoints/superPointNet_{iter}_checkpoint.pth.tar'
    elif mode == 3: 
        config['training']['pretrained'] = f'logs/baselineTrain_sp_deepF_kitti_qtLoss_v3/checkpoints/deepFNet_8000_checkpoint.pth.tar'
        config['training']['pretrained_SP'] = f'logs/baselineTrain_sp_deepF_kitti_qtLoss_v3/checkpoints/superPointNet_{iter}_checkpoint.pth.tar'        
    elif mode == 4:  # 11/2 only train sp
        config['training']['pretrained'] = f'logs/baselineTrain_kittiSp_deepF_end_kittiFLoss_freezeDeepF_v0/checkpoints/deepFNet_8000_checkpoint.pth.tar'
        config['training']['pretrained_SP'] = f'logs/baselineTrain_kittiSp_deepF_end_kittiFLoss_freezeDeepF_v0/checkpoints/superPointNet_{iter}_checkpoint.pth.tar'        


        
    return config


config = change_config(config, iter=1000)    
plot_epipolar_list=[""] # ["mask_conf", "mask_epi_dist_gt"]
plot_corr_list=["random"]  # "all"

# def run_one_sample(config, prefix='', frame_list=[100]):
def run_one_sample(config, sample, prefix='', frame_list=[100]):

    val_agent = Val_pipeline_frontend(config, device=device)
    # val_agent.add_net('deepF', net)
    # val_agent.add_net('superpointNet', net_SP)
    val_agent.load_net_deepF(name='net_deepF')
    val_agent.load_net_SP()
    val_agent.net_toeval() 
    iter_max = 100
    print(sample['scene_name'], sample['frame_ids'])
    val_agent.run_eval(sample, plot_corr_list=plot_corr_list, 
                   plot_epipolar_list=plot_epipolar_list,
                   prefix=prefix, 
                   save=True, title=False)
#         if j >= iter_max:
#             break
    pass

## kitti 
# run_one_sample(config=change_config(config, iter=1000))
# run_one_sample(config=change_config(config, iter=3000))
# run_one_sample(config=change_config(config, iter=5000))
# run_one_sample(config=change_config(config, iter=8000))

# iter_list = [10000, 20000, 30000, 40000, 50000]
# iter_list = [1000, 3000, 5000, 8000]
# iter_list = [0, 1000, 2000, 3000]
iter_list = [0, 200, 400, 600, 800, 1000]

# iter_max = 0
# sample_list = [0, 1426, 1775]

iter_max = 945
sample_list = [945]
from tqdm import tqdm
for j, sample in tqdm(enumerate(val_loader)):
    if j in sample_list:
        for iter in iter_list:
            prefix = f'Sp-Df-f-end-k-freezeDf_s{j}_{iter/1000}k' # 'Sp-Df-fp-end-k'
            run_one_sample(config=change_config(config, iter=iter, mode=4), 
                           sample=sample,
                           frame_list=[100], 
                           prefix=prefix)
    if j > iter_max:
        break
        
## apollo
# run_one_sample(config=change_config(config, iter=6000, mode=1))
# run_one_sample(config=change_config(config, iter=16000, mode=1))
# run_one_sample(config=change_config(config, iter=20000, mode=1))
# run_one_sample(config=change_config(config, iter=40000, mode=1))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from utils.eval_tools import Val_pipeline_frontend

plot_epipolar_list=["mask_conf", "mask_epi_dist_gt"]
plot_corr_list=["all"]
val_agent = Val_pipeline_frontend(config, device=device)
# val_agent.add_net('deepF', net)
# val_agent.add_net('superpointNet', net_SP)
val_agent.load_net_deepF(name='net_deepF')
if config['model']['if_SP']: val_agent.load_net_SP()
val_agent.net_toeval() 

num_list = []
iter_max = 2700
iter_list = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
iter_list = np.arange(27)*100
prefix = 'Si-Df-k' # 'Sp-Df-fp-end-k'
for j, sample in enumerate(val_loader):
#     val_agent.eval_one_sample(sample)
    if j in iter_list:
        print(sample['scene_name'], sample['frame_ids'], sample['img_zoom_xy'])
        logging.info(f"eval {j}")
        val_agent.run_eval(sample, plot_epipolar_list=plot_epipolar_list, 
                           plot_corr_list=plot_corr_list, prefix=prefix, 
                           save=True)
    
    
    if j >= iter_max:
        break

## New functions for evaluation

In [ ]:
iter_max = 0
for j, sample in enumerate(val_loader):
    print(sample['scene_name'], sample['frame_ids'])
    val_agent.run_eval(sample)
    if j >= iter_max:
        break

## Run statistics

In [ ]:
num_corrs = []
from tqdm import tqdm
for j, sample in tqdm(enumerate(train_loader)):
    name='matches_all_unique_nums'
    num_corrs.append(sample[name].numpy())
    if j==200: break
    


In [ ]:
# def print_statistics(arr, name='array'):
#     print(f"{name}: lenght={len(arr)}, mean={arr.mean()}, max={arr.max()}, min={arr.min()}")

from utils.print_tool import print_statistics
print_statistics(np.array(num_corrs).flatten())

In [ ]:
from utils.print_tool import print_dict_attr

print_dict_attr(sample, 'shape')
print(f"{np.array(sample['frame_ids']).T}")

In [ ]:
# a = np.linspace(150, 300, 200)
# b = np.exp(-(a-150)*0.05)
# plt.plot(a, b)
# plt.grid()
# plt.show()

In [ ]:
# ## testing
# # sample["imgs_grey"]
# device

In [ ]:
# weightN_var = torch.tensor(30.)
# weightN_mean = torch.tensor(23.0372)
# tri_depths = torch.tensor(18.5983)

# prob = 1 / torch.sqrt(2*np.pi*weightN_var) * torch.exp(-(tri_depths-weightN_mean)**2 / (2*weightN_var))
# print(prob)

# import matplotlib.pyplot as plt
# import numpy as np
# import scipy.stats as stats
# import math

# mu = 23.0372
# variance = 30.
# sigma = math.sqrt(variance)
# x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
# plt.plot(x, stats.norm.pdf(x, mu, sigma))
# plt.show()

### visualize prediction

In [ ]:
from train_good_utils import (
    get_matches_from_SP,
)
if_SP = config['model']['if_SP']
    

In [ ]:
scores_ori = logits_weights.cpu().numpy().flatten()
import matplotlib.pyplot as plt
plt.hist(scores_ori, 100)
plt.show()

N = scores_ori.shape[0]
print(N)
a_list = []
for idx in range(N):
    selected_corres_idx = np.random.choice(N, 100, p=scores_ori)
    selected_scores = scores_ori[selected_corres_idx]
    a_list.append(selected_scores)
    
a = np.stack(a_list)
plt.hist(a.flatten(), 100)
plt.show()

In [ ]:
%matplotlib inline

errors = []
for topK in range(8, 1000):
    w = outs['weights'].cpu().numpy()
    pts1 = outs['pts1'].cpu().numpy()
    pts2 = outs['pts2'].cpu().numpy()

    scores_ori = logits_weights.cpu().numpy().flatten()
    sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
    # sort_idxes = np.argsort(epi_res[unique_rows_all_idxes])[::-1]
    # scores = scores_ori[unique_rows_all_idxes][sort_idxes
    mask_conf = sort_idxes[:topK]
    # mask_conf = sort_idxes[:sort_idxes.shape[0]]
    # print(scores_ori[unique_rows_all_idxes][mask_conf])

    scores_show = scores_ori[unique_rows_all_idxes[mask_conf]]
    # scores_show = scores_show - 0.8*np.min(scores_show)
    # scores_show = scores_show / np.max(scores_show) * 2. + 0.1
#     plt.figure(figsize=(30, 8))
#     plt.imshow(img1_rgb)
#     plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c=depths_show[mask_conf], s=scores_show*20000+0.5, edgecolors='w', linewidths=2., cmap=cm.plasma)
#     plt.colorbar()
#     plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c='r', s=np.clip(epi_res[unique_rows_all_idxes][mask_conf]*20000+1., 0., 500), marker='+', linewidths=0.)
#     plt.title('Top 10 corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
#     plt.show()

    w_masked = w[:, :, unique_rows_all_idxes[mask_conf]]
    pts1_masked = pts1[:, unique_rows_all_idxes[mask_conf], :]
    pts2_masked = pts2[:, unique_rows_all_idxes[mask_conf], :]


    ## Normalized
    from models.DeepFNet import Fit
    fitt  = Fit(True, True, True)
    out, _= fitt(torch.from_numpy(pts1_masked).cuda(), torch.from_numpy(pts2_masked).cuda(), torch.from_numpy(w_masked).cuda(), if_print=False, matches_good_unique_num=sample['matches_good_unique_nums'])
    F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
    # F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()

    ## NON-Normalized
    # F_my = utils_F._F_from_XY(torch.from_numpy(pts1_masked).cuda().squeeze()[:, :2], torch.from_numpy(pts1_masked).cuda().squeeze()[:, :2], normalize=True)
    # F_est_refit = T2.permute(0,2,1).bmm(F_my.unsqueeze(0).bmm(T1)).squeeze().cpu()

#     utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][mask_conf, :], x2[unique_rows_all_idxes][mask_conf, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='Ours top 20 with largest score points')

    E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
    error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
#     print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

    errors.append(error_Rt_est_ours)
#     print(topK, error_Rt_est_ours)

In [ ]:
plt.figure(figsize=(40, 20))
plt.subplot(211)
plt.plot(list(range(8, 1000)), [error[0] for error in errors])
plt.xlabel('topK')
plt.xticks(list(range(8, 1000, 10)))
plt.ylabel('R error')
plt.ylim([0, 0.5])
plt.grid()
plt.subplot(212)
plt.plot(list(range(8, 1000)), [error[1] for error in errors])
plt.xlabel('topK')
plt.xticks(list(range(8, 1000, 10)))
plt.ylabel('t error')
plt.ylim([0, 5])
plt.grid()
plt.show()

In [ ]:
epi_res = utils_F.compute_epi_residual(outs['pts1'], outs['pts2'], torch.inverse(outs['T2'].permute(0,2,1)) @ F_gt_th.unsqueeze(0).cuda() @ torch.inverse(outs['T1'])).unsqueeze(1)
epi_res = epi_res.cpu().numpy().flatten()
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
# plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], facecolors='none', s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c='r', s=np.clip(epi_res[unique_rows_all_idxes]*20000, 0., 500), marker='+', linewidths=0.)
plt.title('All corres by ours, colored with depth from triangulated GT pose; epi dist in yellow +')
plt.show()

# Triangulate points and reproj error

In [ ]:
# %matplotlib qt
# %matplotlib notebook
# %matplotlib inline
# import matplotlib.pyplot as plt

depth_thres = 200.

scores_show = scores_ori[unique_rows_all_idxes]
scale = np.linalg.norm(delta_Rtij[:3, 3:4]) / np.linalg.norm(M_estW[:3, 3:4])

# depths, est
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
M2 = np.hstack((M_estW[:3, :3], scale * M_estW[:3, 3:4]))
# X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
M2 = delta_Rtij[:3, :]
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]

In [ ]:
x1_reproj = utils_misc.de_homo_np((K @ M1 @ X_tri_homo).T)
x2_reproj = utils_misc.de_homo_np((K @ M2 @ X_tri_homo).T)
depths_show = X_tri[-1, unique_rows_all_idxes]
depths_show = np.clip(depths_show, 0., depth_thres)

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1_reproj[unique_rows_all_idxes, 0], x1_reproj[unique_rows_all_idxes, 1], c=depths_show, s=20, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.show()

In [ ]:
dist = np.linalg.norm(x1_reproj - x1, ord=2, axis=1)
# print(dist)

plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.hist(dist, 100)
plt.subplot(122)
dist_show = np.clip(dist, 0., 2.)
plt.hist(dist_show, 100)
plt.show()


plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=depths_show, s=dist_show[unique_rows_all_idxes]*100+0.5, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c='r', s=np.clip(epi_res[unique_rows_all_idxes]*20000, 0., 500), marker='+', linewidths=0.)
plt.show()

## Offsets

In [ ]:
offsets = outs['offsets'].cpu().numpy().squeeze().T
x1_offsets = offsets[:, :2]
x2_offsets = offsets[:, 2:]
x1_offsets_norm = np.linalg.norm(x1_offsets, 2, 1)

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[:, 0], x1[:, 1], c='b', s=x1_offsets_norm*20, edgecolors='w', linewidths=2.)
plt.show()

In [ ]:
M_estW

In [ ]:
delta_Rtij

In [ ]:
scores_show = scores_ori[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2. + 0.05
scale = np.linalg.norm(delta_Rtij[:3, 3:4]) / np.linalg.norm(M_estW[:3, 3:4])
# depths, est
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R_gt, t_gt))
# M2 = np.hstack((R1, t2))
# M2 = M_estW
M2 = np.hstack((M_estW[:3, :3], scale * M_estW[:3, 3:4]))
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), (x1+x1_offsets).T, (x2+x2_offsets).T)
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]

import matplotlib.cm as cm
depths_show = X_tri[-1, unique_rows_all_idxes]
depths_show = np.clip(depths_show, 0., 200.)
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()

negative_depth_idx = depths_show==0.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][negative_depth_idx], x1[unique_rows_all_idxes, 1][negative_depth_idx], c='k', 
            s=200, marker='X', edgecolors='y', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=depths_show, s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.title('All corres by ours, colored with depth from triangulated EST pose; negative in BLACK X')
plt.show()

epi_res = outs['epi_res_layers'][-1].cpu().numpy().flatten()
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], facecolors='none', s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c='r', s=np.clip(epi_res[unique_rows_all_idxes]*20000+1., 0., 500), marker='+', linewidths=0.)
plt.title('All corres by ours, colored with depth from triangulated EST pose; epi dist in yellow +')
plt.show()

## Compare GT depth and est depth

In [ ]:
print(M_estW)
print(M_opencv)
print(delta_Rtij[:3, :3], delta_Rtij[:3, 3:4] / np.linalg.norm(delta_Rtij[:3, 3:4]))

In [ ]:
# %matplotlib qt
# %matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

depth_thres = 200.

scores_show = scores_ori[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2. + 0.05
scale = np.linalg.norm(delta_Rtij[:3, 3:4]) / np.linalg.norm(M_estW[:3, 3:4])
# scale = 1.
print(scale)

# depths, est
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R_gt, t_gt))
# M2 = np.hstack((R1, t2))
# M2 = M_estW
M2 = np.hstack((M_estW[:3, :3], scale * M_estW[:3, 3:4]))
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]

import matplotlib.cm as cm
depths_show = X_tri[-1, unique_rows_all_idxes]
depths_show = np.clip(depths_show, 0., depth_thres)
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()

negative_depth_idx = depths_show==0.
infinite_depth_idx = depths_show==depth_thres
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][negative_depth_idx], x1[unique_rows_all_idxes, 1][negative_depth_idx], c='k', 
            s=200, marker='X', edgecolors='y', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0][infinite_depth_idx], x1[unique_rows_all_idxes, 1][infinite_depth_idx], c='y', 
            s=200, marker='X', edgecolors='k', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=depths_show, s=scores_show*20000+0.5, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.title('All corres by ours, colored with depth from triangulated EST pose; negative in BLACK X')
plt.show()

epi_res = outs['epi_res_layers'][-1].cpu().numpy().flatten()
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
# plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], facecolors='none', s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.scatter(x1[unique_rows_all_idxes, 0][negative_depth_idx], x1[unique_rows_all_idxes, 1][negative_depth_idx], c='k', 
            s=200, marker='X', edgecolors='y', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0][infinite_depth_idx], x1[unique_rows_all_idxes, 1][infinite_depth_idx], c='y', 
            s=200, marker='X', edgecolors='k', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c='r', s=np.clip(epi_res[unique_rows_all_idxes]*20000+1., 0., 500), marker='+', linewidths=0.)
plt.title('All corres by ours, colored with depth from triangulated EST pose; epi dist in yellow +')
plt.show()

# depths, gt
# scale = 1./np.linalg.norm(delta_Rtij[:3, 3:4])
scale = 1.

M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R_gt, t_gt))
# M2 = np.hstack((R1, t2))
# M2 = M_estW
# M2 = delta_Rtij[:3]
M2 = np.hstack((delta_Rtij[:3, :3], delta_Rtij[:3, 3:4]*scale))
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]

import matplotlib.cm as cm
depth_show = X_tri[-1, unique_rows_all_idxes]
depth_show = np.clip(depths_show, 0., 200.)
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()

negative_depth_idx = depths_show==0.
infinite_depth_idx = depths_show==depth_thres
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][negative_depth_idx], x1[unique_rows_all_idxes, 1][negative_depth_idx], c='k', 
            s=200, marker='X', edgecolors='y', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0][infinite_depth_idx], x1[unique_rows_all_idxes, 1][infinite_depth_idx], c='y', 
            s=200, marker='X', edgecolors='k', linewidths=2.)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=depths_show, s=scores_show*20000, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
# for i in range(x1.shape[0]):
#     plt.text(x1[i, 0], x1[i, 1], str(i), color='r', fontsize=10, fontweight='extra bold')
plt.title('All corres by ours, colored with depth from triangulated GT pose')
plt.show()

In [ ]:
# from models.DeepFNet import Fit
# fitt  = Fit(True, True, True)
# out, _= fitt(outs['pts1'], outs['pts2'], outs['weights'], if_print=False)
# out, residual = fitt(outs['pts1'], outs['pts2'], outs['weights'], if_print=False)

epi_res = utils_F.compute_epi_residual(outs['pts1'], outs['pts2'], torch.inverse(outs['T2'].permute(0,2,1)) @ F_gt_th.unsqueeze(0).cuda() @ torch.inverse(outs['T1'])).unsqueeze(1)
epi_res = epi_res.cpu().numpy().flatten()
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
# plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], facecolors='none', s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c='r', s=np.clip(epi_res[unique_rows_all_idxes]*20000+1., 0., 500), marker='+', linewidths=0.)
plt.title('All corres by ours, colored with depth from triangulated GT pose; epi dist in yellow +')
plt.show()

In [ ]:
F_gt_th

In [ ]:
scale

### Abs and rel depth

In [ ]:
scale = np.linalg.norm(delta_Rtij[:3, 3:4])
# scale = 1.
# depths, est
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R_gt, t_gt))
# M2 = np.hstack((R1, t2))
# M2 = M_estW
M2 = np.hstack((M_estW[:3, :3], scale * M_estW[:3, 3:4]))
print(np.linalg.norm(M2[:3, 3:4]))

X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri_est = X_tri_homo[:3, :]/X_tri_homo[-1, :]
# depths, gt
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R_gt, t_gt))
# M2 = np.hstack((R1, t2))
# M2 = M_estW
# M2 = delta_Rtij[:3]
M2 = np.hstack((delta_Rtij[:3, :3], delta_Rtij[:3, 3:4]))

print(np.linalg.norm(M2[:3, 3:4]))
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri_gt = X_tri_homo[:3, :]/X_tri_homo[-1, :]

# Absolute depth
X_tri = X_tri_gt - X_tri_est

import matplotlib.cm as cm
depths_show = X_tri[-1, unique_rows_all_idxes]
depths_show = np.clip(depths_show, -50., 50.)
scores_show = scores_ori[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2. + 0.4
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=depths_show, s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.bwr)
plt.colorbar()
plt.title('[Absolute depth error] All corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
plt.show()

# Relative depth
X_tri_gt = np.clip(X_tri_gt, 0., 100.)
# X_tri_est = np.clip(X_tri_est, 0., 100.)
X_tri = (X_tri_gt - X_tri_est) / (np.abs(X_tri_gt) + 1e-10)

import matplotlib.cm as cm
depths_show = X_tri[-1, unique_rows_all_idxes]
depths_show = np.clip(depths_show, -0.2, 0.2)
scores_show = scores_ori[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2. + 0.5
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=depths_show, s=scores_show*20000+1., edgecolors='w', linewidths=2., cmap=cm.bwr)
plt.colorbar()
plt.title('[Relative depth error] All corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
plt.show()



# All our scores
scores_show = scores_ori[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=scores_show*20000+1., c='r', edgecolors='w', linewidths=2.)
plt.title('All corres by ours')
plt.show()

# Scores of top ours
N = mask_sample.shape[0]
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
mask_conf = sort_idxes[:N]
scores_show = scores_ori[unique_rows_all_idxes][mask_conf]
# scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], s=scores_show*20000+1., c='r', edgecolors='w', linewidths=2.)
plt.title('Top %d(inlier num by OpenCV) by Ours'%N)
plt.show()

# Scores of top OpenCV
scores_show = scores_ori[mask_sample]
scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[mask_sample, 0], x1[mask_sample, 1], s=scores_show*50, c='r', edgecolors='w', linewidths=2.)
plt.title('Top %d by OpenCV'%N)
plt.show()

# Geo dists
geo_dists = np.sqrt(utils_F._sym_epi_dist(F_gt_th, torch.from_numpy(x1[unique_rows_all_idxes]), torch.from_numpy(x2[unique_rows_all_idxes])).numpy())
# plt.hist(geo_dists, 100)
# plt.show()
geo_dists = np.clip(geo_dists, 0, 10.)
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=geo_dists*50, c='r', edgecolors='w', linewidths=2.)
plt.show()

# All corres
utils_vis.draw_corr(img1_rgb, img2_rgb, x1[unique_rows_all_idxes], x2[unique_rows_all_idxes], linewidth=0., title='All corres')

## Test with subset(top) corres

In [ ]:
%matplotlib inline

errors = []
for topK in range(8, 1000):
    w = outs['weights'].cpu().numpy()
    pts1 = outs['pts1'].cpu().numpy()
    pts2 = outs['pts2'].cpu().numpy()

    scores_ori = logits_weights.cpu().numpy().flatten()
    sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
    # sort_idxes = np.argsort(epi_res[unique_rows_all_idxes])[::-1]
    # scores = scores_ori[unique_rows_all_idxes][sort_idxes
    mask_conf = sort_idxes[:topK]
    # mask_conf = sort_idxes[:sort_idxes.shape[0]]
    # print(scores_ori[unique_rows_all_idxes][mask_conf])

    scores_show = scores_ori[unique_rows_all_idxes[mask_conf]]
    # scores_show = scores_show - 0.8*np.min(scores_show)
    # scores_show = scores_show / np.max(scores_show) * 2. + 0.1
#     plt.figure(figsize=(30, 8))
#     plt.imshow(img1_rgb)
#     plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c=depths_show[mask_conf], s=scores_show*20000+0.5, edgecolors='w', linewidths=2., cmap=cm.plasma)
#     plt.colorbar()
#     plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c='r', s=np.clip(epi_res[unique_rows_all_idxes][mask_conf]*20000+1., 0., 500), marker='+', linewidths=0.)
#     plt.title('Top 10 corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
#     plt.show()

    w_masked = w[:, :, unique_rows_all_idxes[mask_conf]]
    pts1_masked = pts1[:, unique_rows_all_idxes[mask_conf], :]
    pts2_masked = pts2[:, unique_rows_all_idxes[mask_conf], :]


    ## Normalized
    from models.DeepFNet import Fit
    fitt  = Fit(True, True, True)
    out, _= fitt(torch.from_numpy(pts1_masked).cuda(), torch.from_numpy(pts2_masked).cuda(), torch.from_numpy(w_masked).cuda(), if_print=False, matches_good_unique_num=sample['matches_good_unique_nums'])
    F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
    # F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()

    ## NON-Normalized
    # F_my = utils_F._F_from_XY(torch.from_numpy(pts1_masked).cuda().squeeze()[:, :2], torch.from_numpy(pts1_masked).cuda().squeeze()[:, :2], normalize=True)
    # F_est_refit = T2.permute(0,2,1).bmm(F_my.unsqueeze(0).bmm(T1)).squeeze().cpu()

#     utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][mask_conf, :], x2[unique_rows_all_idxes][mask_conf, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='Ours top 20 with largest score points')

    E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
    error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
#     print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

    errors.append(error_Rt_est_ours)
    print(topK, error_Rt_est_ours)

%matplotlib notebook
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(40, 20))
plt.subplot(211)
plt.plot(list(range(8, 1000)), [error[0] for error in errors])
plt.xlabel('topK')
plt.xticks(list(range(8, 1000, 10)))
plt.ylabel('R error')
plt.ylim([0, 5])
plt.grid()
plt.subplot(212)
plt.plot(list(range(8, 1000)), [error[1] for error in errors])
plt.xlabel('topK')
plt.xticks(list(range(8, 1000, 10)))
plt.ylabel('t error')
plt.ylim([0, 5])
plt.grid()
plt.show()



In [ ]:
%matplotlib inline

w = outs['weights'].cpu().numpy()
pts1 = outs['pts1'].cpu().numpy()
pts2 = outs['pts2'].cpu().numpy()

scores_ori = logits_weights.cpu().numpy().flatten()
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
# sort_idxes = np.argsort(epi_res[unique_rows_all_idxes])[::-1]
# scores = scores_ori[unique_rows_all_idxes][sort_idxes
mask_conf = sort_idxes[:10]
# mask_conf = sort_idxes[:sort_idxes.shape[0]]
# print(scores_ori[unique_rows_all_idxes][mask_conf])

scores_show = scores_ori[unique_rows_all_idxes[mask_conf]]
# scores_show = scores_show - 0.8*np.min(scores_show)
# scores_show = scores_show / np.max(scores_show) * 2. + 0.1
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c=depths_show[mask_conf], s=scores_show*20000+0.5, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c='r', s=np.clip(epi_res[unique_rows_all_idxes][mask_conf]*20000+1., 0., 500), marker='+', linewidths=0.)
plt.title('Top 10 corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
plt.show()

w_masked = w[:, :, unique_rows_all_idxes[mask_conf]]
pts1_masked = pts1[:, unique_rows_all_idxes[mask_conf], :]
pts2_masked = pts2[:, unique_rows_all_idxes[mask_conf], :]


## Normalized
from models.DeepFNet import Fit
fitt  = Fit(True, True, True)
out, _= fitt(torch.from_numpy(pts1_masked).cuda(), torch.from_numpy(pts2_masked).cuda(), torch.from_numpy(w_masked).cuda(), if_print=False, matches_good_unique_num=sample['matches_good_unique_nums'])
F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
# F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()

## NON-Normalized
# F_my = utils_F._F_from_XY(torch.from_numpy(pts1_masked).cuda().squeeze()[:, :2], torch.from_numpy(pts1_masked).cuda().squeeze()[:, :2], normalize=True)
# F_est_refit = T2.permute(0,2,1).bmm(F_my.unsqueeze(0).bmm(T1)).squeeze().cpu()

utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][mask_conf, :], x2[unique_rows_all_idxes][mask_conf, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='Ours top 20 with largest score points')

E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))



In [ ]:
logits_weights = outs['weights'].cpu().numpy()
pts1 = outs['pts1'].cpu().numpy()
pts2 = outs['pts2'].cpu().numpy()

logits_weights[0, 0, 187] = 0.

scores_ori = logits_weights.flatten()
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
# sort_idxes = np.argsort(epi_res[unique_rows_all_idxes])[::-1]
# scores = scores_ori[unique_rows_all_idxes][sort_idxes
# mask_conf = sort_idxes[:300]
mask_conf = sort_idxes[:sort_idxes.shape[0]]
# print(scores_ori[unique_rows_all_idxes][mask_conf])


scores_show = scores_ori[unique_rows_all_idxes[mask_conf]]
# scores_show = scores_show - 0.8*np.min(scores_show)
scores_show = scores_show / np.max(scores_show) * 2. + 0.1
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c=depths_show[mask_conf], s=scores_show*50, edgecolors='w', linewidths=2., cmap=cm.bwr)
plt.colorbar()
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c='r', s=np.clip(epi_res[unique_rows_all_idxes][mask_conf]*20000+1., 0., 500), marker='+', linewidths=0.)
plt.title('Top 10 corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
plt.show()

w_masked = logits_weights
pts1_masked = pts1
pts2_masked = pts2

out, _= fitt(torch.from_numpy(pts1_masked).cuda(), torch.from_numpy(pts2_masked).cuda(), torch.from_numpy(w_masked).cuda(), if_print=False)
F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
# F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()

utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][:30, :], x2[unique_rows_all_idxes][:30, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='Ours top 20 with largest score points')

E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))



## Filter points

In [ ]:
%matplotlib inline
# %matplotlib notebook

scale = np.linalg.norm(delta_Rtij[:3, 3:4])
# scale = np.linalg.norm(delta_Rtij[:3, 3:4]) / np.linalg.norm(M_estW[:3, 3:4])

# depths, est
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = M_estW
M2 = np.hstack((M_estW[:3, :3], scale * M_estW[:3, 3:4]))
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri_est = X_tri_homo[:3, :]/X_tri_homo[-1, :]

# depths, gt
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = delta_Rtij[:3]
M2 = np.hstack((delta_Rtij[:3, :3], delta_Rtij[:3, 3:4]))
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri_gt = X_tri_homo[:3, :]/X_tri_homo[-1, :]

X_tri = X_tri_gt - X_tri_est
X_tri_rel = (X_tri_gt - X_tri_est) / (np.abs(X_tri_gt) + 1e-10)

# depth_diff = X_tri[-1, :]
depth_diff = X_tri_rel[-1, :]
# depth_diff_idx = np.abs(depth_diff) > 0.15
depth_diff_idx = X_tri_est[-1, :] <= 0.
depth_diff_idx = np.logical_or(X_tri_est[-1, :] <= 0., X_tri_est[-1, :] >200.)
# depth_diff_idx = scores_ori < 0.001


import matplotlib.cm as cm
depths_show = X_tri_rel[-1, unique_rows_all_idxes]
depths_show = np.clip(depths_show, -0.4, 0.4)
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()


from models.DeepFNet import Fit
fitt  = Fit(True, True, True)
out, _= fitt(outs['pts1'], outs['pts2'], outs['weights'], if_print=False)
F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
# F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()
utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][mask_conf, :], x2[unique_rows_all_idxes][mask_conf, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='[Before removal] Ours top 20 with largest score points')
E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('[Before removal] Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))


w = outs['weights'].cpu().numpy()
plt.hist(w.flatten(), 100)
plt.show()
w[:, :, depth_diff_idx] = 0.
plt.hist(w.flatten(), 100)
plt.show()

# w[:, :, :] = 0.
out, _= fitt(outs['pts1'], outs['pts2'], torch.from_numpy(w).cuda(), if_print=False)
F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
# F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()

scores_after = w.flatten()
sort_idxes_after = np.argsort(scores_after[unique_rows_all_idxes])[::-1]
mask_conf_after = sort_idxes_after[:20]

utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][mask_conf_after, :], x2[unique_rows_all_idxes][mask_conf_after, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='[After removal] Ours top 20 with largest score points')

E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('[After removal] Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

keep_idx = np.logical_not(depth_diff_idx)
depths_show = X_tri_est[-1, :]
depths_show = np.clip(depths_show, 0., 800.)

scores_show = scores_after
# scores_show = scores_show / np.max(scores_show) * 2. + 0.0
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[keep_idx, 0], x1[keep_idx, 1], c=depths_show[keep_idx], s=scores_show[keep_idx]*20000+1, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.title('[After removal] All corres by ours, colored with EST depth from triangulated GT and EST pose; size is the score')
plt.show()

# Test different weight patterns

In [ ]:
w = outs['weights'].cpu().numpy()
pts1 = outs['pts1'].cpu().numpy()
pts2 = outs['pts2'].cpu().numpy()

scores_ori = logits_weights.cpu().numpy().flatten()
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
# scores = scores_ori[unique_rows_all_idxes][sort_idxes
mask_conf = sort_idxes[:9]
print(scores_ori[unique_rows_all_idxes][mask_conf])

scores_show = scores_ori[unique_rows_all_idxes[mask_conf]]
scores_show = scores_show - 0.8*np.min(scores_show)
scores_show = scores_show / np.max(scores_show) * 2. + 0.0
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], c=depths_show[mask_conf], s=scores_show*50, edgecolors='w', linewidths=2., cmap=cm.bwr)
plt.colorbar()
plt.title('Top 10 corres by ours, colored with depth DIFFERENCE from triangulated GT and EST pose; size is the score')
plt.show()

w_masked = w[:, :, unique_rows_all_idxes[mask_conf]]
pts1_masked = pts1[:, unique_rows_all_idxes[mask_conf], :]
pts2_masked = pts2[:, unique_rows_all_idxes[mask_conf], :]

out, _= fitt(torch.from_numpy(pts1_masked).cuda(), torch.from_numpy(pts2_masked).cuda(), torch.from_numpy(w_masked).cuda(), if_print=False)
F_est_refit = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1']))[0].cpu()
# F_est_refit = outs['T2'].permute(0,2,1).bmm(outs['F_est'].bmm(outs['T1']))[0].cpu()

utils_vis.show_epipolar_rui_gtEst(x1[unique_rows_all_idxes][mask_conf, :], x2[unique_rows_all_idxes][mask_conf, :], img1_rgb, img2_rgb, F_gt, F_est_refit, im_shape=im_shape, title_append='Ours top 20 with largest score points')

E_ests_refit = Ks.transpose(1, 2) @ F_est_refit.cuda() @ Ks
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_ests_refit.cpu().numpy(), E_gt, F_est_refit.cpu().numpy(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))



# Lidar points

In [ ]:
# scale= 1./np.linalg.norm(delta_Rtij[:3, 3:4])
scale = 1.
print(scale)

X = Xs[1].squeeze().numpy() * scale

delta_Rtij_camid = np.hstack((delta_Rtij[:3, :3], scale*delta_Rtij[:3, 3:4]))
param_list = [K, img1_rgb.shape]

N_velo = 100000
choice_all = utils_misc.crop_or_pad_choice(X.shape[0], N_velo, shuffle=True)
sample_Xj = X[choice_all]
# sample_Xj = X
delta_Rtij_camid = np.linalg.inv(utils_misc.Rt_pad(delta_Rtij_camid))
_, xi = utils_vis.reproj_and_scatter(utils_misc.Rt_depad(delta_Rtij_camid), sample_Xj, img1_rgb, visualize=True, title_appendix='j to i', param_list=param_list, debug=False, s=10)
_, xj = utils_vis.reproj_and_scatter(utils_misc.identity_Rt(), sample_Xj, img2_rgb, visualize=True, title_appendix='j to j', param_list=param_list, debug=False, s=10)

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi[:, 0], xi[:, 1], c='r', s=50, edgecolors='w', linewidths=2., cmap=cm.bwr)
plt.show()

In [ ]:
pts1_virt_ori.device

In [ ]:
# print(utils_misc.homo_np(x2) @ F_gt @ utils_misc.homo_np(x1).T)
# print(utils_misc.homo_np(xj) @ F_gt @ utils_misc.homo_np(xi).T)
pts1_velo_ori = torch.from_numpy(utils_misc.homo_np(xi)).cuda().unsqueeze(0)
pts2_velo_ori = torch.from_numpy(utils_misc.homo_np(xj)).cuda().unsqueeze(0)

losses = utils_F.compute_epi_residual(pts1_velo_ori, pts2_velo_ori, sample['F'].cuda(), 0.02) #- res.mean()

print(losses.cpu().numpy())

## [Optional] Fake lidar motion

In [ ]:
from pyquaternion import Quaternion

q_fake = Quaternion(axis=[0., 1., 0.], degrees=0.)
R_fake = q_fake.rotation_matrix
# t_fake = np.zeros((3, 1))
t_fake = np.array([0., 0.00, -1.]).reshape(3, 1)
# t_fake = np.array([0.1, 0., 0.]).reshape(3, 1)

X = Xs[0].squeeze().numpy()
print(X.shape)
N_velo = 200
choice_all = utils_misc.crop_or_pad_choice(X.shape[0], N_velo, shuffle=True)
sample_X = X[choice_all]

import matplotlib.cm as cm
delta_Rtij_camid = np.hstack((R_fake, t_fake))
param_list = [K, img1_rgb.shape]
_, xi = utils_vis.reproj_and_scatter(utils_misc.identity_Rt(), sample_X, img1_rgb, visualize=True, title_appendix='j to i', param_list=param_list, debug=False, s=10)
_, xj = utils_vis.reproj_and_scatter(delta_Rtij_camid, sample_X, img1_rgb, visualize=True, title_appendix='i to j', param_list=param_list, debug=False, s=10)

N_velo = xi.shape[0]
E_gt, F_gt = utils_F.E_F_from_Rt_np(R_fake, t_fake, K)

delta_Rtij_inv = np.linalg.inv(utils_misc.Rt_pad(np.hstack((R_fake, t_fake))))

In [ ]:
F_gt

### Visualize: Perfect points

In [ ]:
xi_th = torch.from_numpy(xi).float()
xj_th = torch.from_numpy(xj).float()

from models.DeepFNet import NormalizeAndExpand_K, Fit
norm_K = NormalizeAndExpand_K(False, True)
pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]

fit_velo  = Fit(False, True, True)
out_velo, _= fit_velo(pts1.cuda().transpose(1, 2), pts2.cuda().transpose(1, 2), torch.ones((1, 1, N_velo)).float().cuda(), if_print=False)
F_est_refit = T2.permute(0,2,1).bmm(out_velo.cpu().bmm(T1)).squeeze()
print(F_est_refit.numpy())
utils_vis.show_epipolar_rui_gtEst(xj, xi, img2_rgb, img2_rgb, F_gt.T, F_est_refit.t(), im_shape=im_shape, title_append='')

# Triangulation
E_est_refit = utils_F._F_to_E(F_est_refit, K_th)
R2s, t2s, M2s = utils_F._get_M2s(E_est_refit)
print(delta_Rtij[:3])
print(R2s[0].numpy(), R2s[1].numpy())
print(t2s[0].numpy(), t2s[1].numpy())
# M2_list, error_Rt, Rt_cam = utils_F._E_to_M(E_est_refit, K, xi, xj, inlier_mask=None, delta_Rt_gt=delta_Rtij_inv)
        
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
M2 = np.hstack((R2s[1].numpy(), t2s[1].numpy()))

X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), xi.T, xj.T)
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]
import matplotlib.cm as cm
depths_show = X_tri[-1, :]
# depths_show = np.clip(depths_show, 0., 80.)
plt.figure(figsize=(30, 2))
plt.hist(depths_show, 200)
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi[:, 0], xi[:, 1], c=depths_show, s=100., edgecolors='w', linewidths=2., cmap=cm.plasma)
for i in range(N_velo):
    plt.text(xi[i, 0], xi[i, 1]-10, str(i), color='r', fontsize=20, fontweight='extra bold')
plt.colorbar()
# plt.title('All corres by ours, colored with depth from triangulated EST pose')
plt.show()

In [ ]:
print(F_est_refit.numpy() / F_est_refit.numpy()[2, 2])
print(F_gt / F_gt[2, 2])

In [ ]:
pts1_eval = (T1 @ pts1_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
pts2_eval = (T2 @ pts2_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
losses = utils_F.compute_epi_residual(pts1_eval, pts2_eval, torch.inverse(T2.permute(0,2,1)) @ F_est_refit @ torch.inverse(T1), loss_params['clamp_at'])
print(losses.mean().numpy())

# E_est_refit = K_th.t() @ F_est_refit @ K_th
E_est_refit = K_th.t() @ F_gt_th @ K_th

R2s, t2s, M2s = utils_F._get_M2s(E_est_refit)
print('---gt', delta_Rtij[:3, :3], delta_Rtij[:3, 3:4]/np.linalg.norm(delta_Rtij[:3, 3:4]))
print('---est R', R2s[0].numpy(), R2s[1].numpy())
print('---est t', t2s[0].numpy(), t2s[1].numpy())

# M_estW, error_Rt_est_ours = utils_F.goodCorr_eval_nondecompose(xi, xj, E_est_refit.numpy().astype(np.float64), delta_Rtij_inv, K, None, Rt_scene_list=[R2s[1].numpy(), t2s[1].numpy()])
M_estW, error_Rt_est_ours = utils_F.goodCorr_eval_nondecompose(xi, xj, E_est_refit.numpy().astype(np.float64), delta_Rtij_inv, K, None, Rt_scene_list=None)

print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

print('-=-- Est R, t')
print(M_estW)

### Add some noise to some point(s)

#### Inperfect points: sample SIFT kps

In [ ]:
N_velo = 200

scores_ori = logits_weights.cpu().numpy().flatten()
plt.hist(scores_ori, 100)
plt.show()
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
scores = scores_ori[unique_rows_all_idxes][sort_idxes]
mask_conf = sort_idxes[:N_velo]

plt.hist(scores_ori[unique_rows_all_idxes][mask_conf], 100)
plt.show()

xi = x1[unique_rows_all_idxes][mask_conf]
xj = x2[unique_rows_all_idxes][mask_conf]


In [ ]:
min_thres = -2.
max_thres = 2.

xi_noise = xi.copy()
xj_noise = xj.copy()
noise_idx = 5
while True:
    noise = np.random.normal(size=(1, 2))
    if not(noise[0, 0] < min_thres or noise[0, 1] < min_thres or noise[0, 0] > max_thres or noise[0, 1] > max_thres):
        break
print(noise)
# xi_noise[noise_idx] = xi_noise[noise_idx] + noise

xi_th = torch.from_numpy(xi_noise).float()
xj_th = torch.from_numpy(xj_noise).float()

from models.DeepFNet import NormalizeAndExpand_K, Fit
norm_K = NormalizeAndExpand_K(False, True)
pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]

fit_velo  = Fit(False, True, True)
out_velo, _= fit_velo(pts1.cuda().transpose(1, 2), pts2.cuda().transpose(1, 2), torch.ones((1, 1, N_velo)).float().cuda(), if_print=False)
F_est_refit_noise = T2.permute(0,2,1).bmm(out_velo.cpu().bmm(T1)).squeeze()
# print(out_velo.cpu().squeeze().numpy() / out_velo.cpu().squeeze().numpy()[2, 2])

# F_my = utils_F._F_from_XY(pts1.squeeze().t()[:, :2], pts2.squeeze().t()[:, :2], normalize=True)
# # print(F_my.numpy() / F_my.numpy()[2, 2])
# F_est_refit_noise = T2.permute(0,2,1).bmm(F_my.unsqueeze(0).bmm(T1)).squeeze()

# Errors
pts1_eval = (T1 @ pts1_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
pts2_eval = (T2 @ pts2_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
losses = utils_F.compute_epi_residual(pts1_eval, pts2_eval, torch.inverse(T2.permute(0,2,1)) @ F_est_refit_noise @ torch.inverse(T1), loss_params['clamp_at'])
print(losses.mean().numpy())

E_est_refit = utils_F._F_to_E(F_est_refit_noise, K_th)
M_estW, error_Rt_est_ours = utils_F.goodCorr_eval_nondecompose(xi_noise, xj_noise, E_est_refit.numpy().astype(np.float64), delta_Rtij_inv, K, None)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

# Visualization
# utils_vis.show_epipolar_rui_gtEst(xj, xi, img2_rgb, img1_rgb, F_gt.T, F_est_refit.t(), im_shape=im_shape, title_append='Noiseless')
utils_vis.show_epipolar_rui_gtEst(xj_noise, xi_noise, img2_rgb, img1_rgb, F_gt.T, F_est_refit_noise.t(), im_shape=im_shape, title_append='With Noise', emphasis_idx=[noise_idx], label_text=True)

# Triangulation
# R2s, t2s, M2s = utils_F._get_M2s(E_est_refit)
# print(delta_Rtij[:3])
# print(R2s[0].numpy(), R2s[1].numpy())
# print(t2s[0].numpy(), t2s[1].numpy())
# # M2_list, error_Rt, Rt_cam = utils_F._E_to_M(E_est_refit, K, xi, xj, inlier_mask=None, delta_Rt_gt=delta_Rtij_inv)
        
# M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R2s[1].numpy(), t2s[1].numpy()))

# X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), xi_noise.T, xj_noise.T)
# X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]
# import matplotlib.cm as cm
# depths_show = X_tri[-1, :]
# # depths_show = np.clip(depths_show, 0., 80.)
# # plt.figure(figsize=(30, 2))
# # plt.hist(depths_show, 200)
# # plt.show()

# plt.figure(figsize=(30, 8))
# plt.imshow(img1_rgb)
# plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c=depths_show, s=100., edgecolors='w', linewidths=2., cmap=cm.plasma)
# for i in range(N_velo):
#     plt.text(xi_noise[i, 0], xi_noise[i, 1]-10, str(i), color='r', fontsize=20, fontweight='extra bold')
# plt.colorbar()
# # plt.title('All corres by ours, colored with depth from triangulated EST pose')

# plt.show()

### [Where are good points that if moves a bit gives least errors?] Generate noises and get error stats

In [ ]:
delta_Rtij

In [ ]:
delta_Rtij_inv

In [ ]:
noise_idx  = 10

plt.figure()
min_thres = -2.
max_thres = 2.
scale = 1.
for tries in range(1000):
#     print(noise_idx, tries)
    xi_noise = xi.copy()
    xj_noise = xj.copy()
    while True:
        noise = np.random.normal(scale=scale, size=(1, 2))
        if not(noise[0, 0] < min_thres or noise[0, 1] < min_thres or noise[0, 0] > max_thres or noise[0, 1] > max_thres):
            break
            
    xi_noise[noise_idx] = xi_noise[noise_idx] + noise
    plt.scatter(xi_noise[noise_idx][0], xi_noise[noise_idx][1])

plt.show()

In [ ]:
from models.DeepFNet import NormalizeAndExpand_K, Fit

error_list = {}
for noise_idx in range(N_velo):
    error_list[str(noise_idx)] = [[], [], []]

for noise_idx in range(N_velo):
    for tries in range(1---):
        print(noise_idx, tries)
        xi_noise = xi.copy()
        xj_noise = xj.copy()
        
        while True:
            noise = np.random.normal(scale=scale, size=(1, 2))
            if not(noise[0, 0] < min_thres or noise[0, 1] < min_thres or noise[0, 0] > max_thres or noise[0, 1] > max_thres):
                break
            
        xi_noise[noise_idx] = xi_noise[noise_idx] + noise
        
        weight = torch.ones((1, 1, N_velo))
#         weight[:, :, noise_idx] = 0.

        xi_th = torch.from_numpy(xi_noise).float()
        xj_th = torch.from_numpy(xj_noise).float()

        norm_K = NormalizeAndExpand_K(False, True)
        pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]

        fit_velo  = Fit(False, True, True)
        out_velo, _= fit_velo(pts1.cuda().transpose(1, 2), pts2.cuda().transpose(1, 2), weight.float().cuda(), if_print=False)
        F_est_refit_noise = T2.permute(0,2,1).bmm(out_velo.cpu().bmm(T1)).squeeze()
#         F_my = utils_F._F_from_XY(pts1.squeeze().t()[:, :2], pts2.squeeze().t()[:, :2], normalize=True)
#         F_est_refit_noise = T2.permute(0,2,1).bmm(F_my.unsqueeze(0).bmm(T1)).squeeze()

        # Errors
        pts1_eval = (T1 @ pts1_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
        pts2_eval = (T2 @ pts2_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
        losses = utils_F.compute_epi_residual(pts1_eval, pts2_eval, torch.inverse(T2.permute(0,2,1)) @ F_est_refit_noise @ torch.inverse(T1), loss_params['clamp_at'])
#         print(losses.mean().numpy())

        E_est_refit = utils_F._F_to_E(F_est_refit_noise, K_th)
        M_estW, error_Rt_est_ours = utils_F.goodCorr_eval_nondecompose(xi_noise, xj_noise, E_est_refit.numpy().astype(np.float64), delta_Rtij_inv, K, None, Rt_scene_list=None)
#         print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))
        
        error_list[str(noise_idx)][0].append(error_Rt_est_ours[0])
        error_list[str(noise_idx)][1].append(error_Rt_est_ours[1])

In [ ]:
error_stats_dict = {}
for noise_idx in range(N_velo):
    all_R_errors = error_list[str(noise_idx)][0]
    all_t_errors = error_list[str(noise_idx)][1]
    error_stats_dict[str(noise_idx)] = {'R_mean': np.mean(all_R_errors), 'R_std': np.std(all_R_errors), \
                                        't_mean': np.mean(all_t_errors), 't_std': np.std(all_t_errors), \
                                       'R_med': np.median(all_R_errors), 't_med': np.median(all_t_errors)}

R_means = [error_stats_dict[str(idx)]['R_mean'] for idx in range(N_velo)]
t_means = [error_stats_dict[str(idx)]['t_mean'] for idx in range(N_velo)]
R_stds = [error_stats_dict[str(idx)]['R_std'] for idx in range(N_velo)]
t_stds = [error_stats_dict[str(idx)]['t_std'] for idx in range(N_velo)]
R_meds = [error_stats_dict[str(idx)]['R_med'] for idx in range(N_velo)]
t_meds = [error_stats_dict[str(idx)]['t_med'] for idx in range(N_velo)]

scale = 0.5
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(R_meds)*10000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('R_med 10X')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(t_meds)*1000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('t_med')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(R_means)*10000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('R_mean 10X')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(t_means)*1000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('t_mean')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(R_stds)*10000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('R_std 10X')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(t_stds)*1000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('t_std')
plt.show()

### Noise to all points

In [ ]:
xi_th = torch.from_numpy(xi).float()
xj_th = torch.from_numpy(xj).float()
from models.DeepFNet import NormalizeAndExpand_K, Fit
norm_K = NormalizeAndExpand_K(False, True)
pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]


min_thres = -1.5
max_thres = 1.5

xi_noise = xi.copy()
xj_noise = xj.copy()
# noise_idx = 12
# while True:
noise = np.clip(np.random.normal(scale=1., size=(N_velo, 2)), min_thres, max_thres)
#     if not(noise[0, 0] < min_thres or noise[0, 1] < min_thres or noise[0, 0] > max_thres or noise[0, 1] > max_thres):
#         break
xi_noise = xi_noise + noise

xi_th = torch.from_numpy(xi_noise).float()
xj_th = torch.from_numpy(xj_noise).float()

from models.DeepFNet import NormalizeAndExpand_K, Fit
norm_K = NormalizeAndExpand_K(False, True)
pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]

In [ ]:
delta_Rtij_inv

In [ ]:
error_list = []
for idx in range(10000):
    print(idx)
    xi_th = torch.from_numpy(xi).float()
    xj_th = torch.from_numpy(xj).float()
    from models.DeepFNet import NormalizeAndExpand_K, Fit
    norm_K = NormalizeAndExpand_K(False, True)
    pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]

    dists = torch.norm(pts1[:, :2, :], p=2, dim=1, keepdim=True)
#     sort_idxes = np.argsort(dists.numpy().flatten())[::-1] # largest
    sort_idxes = np.argsort(dists.numpy().flatten()) # smallest
    print(dists.numpy()[:, :, sort_idxes[:10]])

    xi_noise = xi.copy()
    xj_noise = xj.copy()

    N_noise = 200
    min_thres = -50.
    max_thres = 50.
    noise_i = np.clip(np.random.normal(scale=3, size=(N_noise, 2)), min_thres, max_thres)
    noise_j = np.clip(np.random.normal(scale=3, size=(N_noise, 2)), min_thres, max_thres)
#     plt.scatter(noise[:, 0], noise[:, 1])
#     plt.show()

    xi_noise[sort_idxes[:N_noise]] = xi_noise[sort_idxes[:N_noise]] + noise_i
    xj_noise[sort_idxes[:N_noise]] = xj_noise[sort_idxes[:N_noise]] + noise_j
#     xi_noise = xi_noise + noise_i
#     xj_noise = xj_noise + noise_j

    xi_th = torch.from_numpy(xi_noise).float()
    xj_th = torch.from_numpy(xj_noise).float()
    pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]





#     weights = torch.ones((1, 1, N_velo))
    # weights = torch.nn.functional.normalize(torch.norm(pts1[:, :2, :], p=2, dim=1, keepdim=True), p=1, dim=2)
    # weights = -weights + torch.max(weights)
    # weights = torch.nn.functional.normalize(1./torch.norm(pts1[:, :2, :], p=2, dim=1, keepdim=True), p=1, dim=2)
    
    weights = np.zeros((1, 1, N_velo))
    weights[:, :, sort_idxes[:N_noise]] = 1.
    weights = torch.from_numpy(weights).float()

    # weights[:, :, [14, 17, 2]] = 0.
    # print(weights)

    # fit_velo  = Fit(False, True, True)
    # out_velo, _= fit_velo(pts1.cuda().transpose(1, 2), pts2.cuda().transpose(1, 2), weights.float().cuda(), if_print=False)
    # F_est_refit_noise = T2.permute(0,2,1).bmm(out_velo.cpu().bmm(T1)).squeeze()
    # print(out_velo.cpu().squeeze().numpy() / out_velo.cpu().squeeze().numpy()[2, 2])

    F_my = utils_F._F_from_XY(pts1.squeeze().t()[:, :2], pts2.squeeze().t()[:, :2], torch.diag(weights.float().squeeze()), normalize=True)
    # print(F_my.numpy() / F_my.numpy()[2, 2])
    F_est_refit_noise = T2.permute(0,2,1).bmm(F_my.unsqueeze(0).bmm(T1)).squeeze()

    # Errors
    pts1_eval = (T1 @ pts1_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
    pts2_eval = (T2 @ pts2_virt_ori.permute(0,2,1).cpu()).permute(0,2,1)
    losses = utils_F.compute_epi_residual(pts1_eval, pts2_eval, torch.inverse(T2.permute(0,2,1)) @ F_est_refit_noise @ torch.inverse(T1), loss_params['clamp_at'])
    print(losses.mean().numpy())

    E_est_refit = utils_F._F_to_E(F_est_refit_noise, K_th)
    M_estW, error_Rt_est_ours = utils_F.goodCorr_eval_nondecompose(xi_noise, xj_noise, E_est_refit.numpy().astype(np.float64), delta_Rtij_inv, K, None)
    print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))
    error_list.append(error_Rt_est_ours)

    # Visualization
    # utils_vis.show_epipolar_rui_gtEst(xj, xi, img2_rgb, img1_rgb, F_gt.T, F_est_refit.t(), im_shape=im_shape, title_append='Noiseless')
#     utils_vis.show_epipolar_rui_gtEst(xj_noise, xi_noise, img2_rgb, img1_rgb, F_gt.T, F_est_refit_noise.t(), im_shape=im_shape, title_append='With Noise', label_text=False)

    # Triangulation
    # R2s, t2s, M2s = utils_F._get_M2s(E_est_refit)
    # print(delta_Rtij[:3])
    # print(R2s[0].numpy(), R2s[1].numpy())
    # print(t2s[0].numpy(), t2s[1].numpy())
    # # M2_list, error_Rt, Rt_cam = utils_F._E_to_M(E_est_refit, K, xi, xj, inlier_mask=None, delta_Rt_gt=delta_Rtij_inv)

    # M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
    # M2 = np.hstack((R2s[1].numpy(), t2s[1].numpy()))

    # X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), xi_noise.T, xj_noise.T)
    # X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]
    # import matplotlib.cm as cm
    # depths_show = X_tri[-1, :]
    # # depths_show = np.clip(depths_show, 0., 80.)
    # # plt.figure(figsize=(30, 2))
    # # plt.hist(depths_show, 200)
    # # plt.show()
    
    if idx==0:
        plt.figure(figsize=(30, 8))
        plt.imshow(img1_rgb)
        plt.scatter(xi_noise[:, 0], xi_noise[:, 1], s=weights.squeeze().numpy()*100., edgecolors='w', linewidths=2., cmap=cm.plasma)
    #     for i in range(N_velo):
    #         plt.text(xi_noise[i, 0], xi_noise[i, 1]-10, str(i), color='r', fontsize=20, fontweight='extra bold')
    #     plt.colorbar()
        plt.title('All corres by ours, colored with depth from triangulated EST pose')

        plt.show()

In [ ]:
plt.scatter(noise_i[:, 0], noise_i[:, 1])
plt.show()

In [ ]:
err_rs = [err[0] for err in error_list]
err_ts = [err[1] for err in error_list]

print(np.mean(err_rs), np.median(err_rs), np.std(err_rs))
print(np.mean(err_ts), np.median(err_ts), np.std(err_ts))

## Noise to each point; measure epi_dist

In [ ]:
F_gt

In [ ]:
from models.DeepFNet import NormalizeAndExpand_K, Fit

error_list = {}
for noise_idx in range(N_velo):
    error_list[str(noise_idx)] = []
    
min_thres = -4.
max_thres = 4.
scale = 2.

for noise_idx in range(N_velo):
    for tries in range(100):
        print(noise_idx, tries)
        xi_noise = xi.copy()
        xj_noise = xj.copy()
        
        while True:
            noise = np.random.normal(scale=scale, size=(1, 2))
            if not(noise[0, 0] < min_thres or noise[0, 1] < min_thres or noise[0, 0] > max_thres or noise[0, 1] > max_thres):
                break
            
        xi_noise[noise_idx] = xi_noise[noise_idx] + noise
        
        while True:
            noise = np.random.normal(scale=scale, size=(1, 2))
            if not(noise[0, 0] < min_thres or noise[0, 1] < min_thres or noise[0, 0] > max_thres or noise[0, 1] > max_thres):
                break
            
        xj_noise[noise_idx] = xj_noise[noise_idx] + noise
        
        weight = torch.ones((1, 1, N_velo))
#         weight[:, :, noise_idx] = 0.

        xi_th = torch.from_numpy(xi_noise).float()
        xj_th = torch.from_numpy(xj_noise).float()

        norm_K = NormalizeAndExpand_K(False, True)
        pts1, pts2, T1, T2 = norm_K(torch.cat((xi_th, xj_th), 1).unsqueeze(0), sample['K_inv']) # pts: [b, N, 2] # \in [-1, 1]
        
        F_gt_normalize = torch.inverse(T2.permute(0,2,1)) @ F_gt_th @ torch.inverse(T1)
        epi_res = utils_F.compute_epi_residual(pts1.transpose(1, 2), pts2.transpose(1, 2), F_gt_normalize).unsqueeze(1)
        
        error_list[str(noise_idx)].append(epi_res.squeeze()[noise_idx].numpy())

In [ ]:
error_stats_dict = {}
for noise_idx in range(N_velo):
    all_epis = error_list[str(noise_idx)]
    error_stats_dict[str(noise_idx)] = {'epi_mean': np.mean(all_epis), 'epi_std': np.std(all_epis), 'epi_med': np.median(all_epis)}

epi_means = [error_stats_dict[str(idx)]['epi_mean'] for idx in range(N_velo)]
epi_stds = [error_stats_dict[str(idx)]['epi_std'] for idx in range(N_velo)]
epi_meds = [error_stats_dict[str(idx)]['epi_med'] for idx in range(N_velo)]

scale = 5
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(epi_meds)*10000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('epi_med 10X')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(epi_means)*10000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('epi_mean 10X')
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(xi_noise[:, 0], xi_noise[:, 1], c='r', s=np.array(epi_stds)*10000*scale, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.title('epi_std 10X')
plt.show()

## From reproj lidar

In [ ]:
x1_lidar = sample['pts1_velo'].squeeze().numpy()[:, :2]
x2_lidar = sample['pts2_velo'].squeeze().numpy()[:, :2]
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1_lidar[:, 0], x1_lidar[:, 1], c='r')
plt.title('Reproj lidr points')
plt.show()

## 4 poses

In [ ]:
print(sample['scene_name'], sample['frame_ids'])

In [ ]:
np.set_printoptions(precision=8, suppress=True)
delta_P = sample['releative_scene_poses'][1].numpy()[0]
delta_R = delta_P[:3, :3]
print(delta_R)
print(np.linalg.det(delta_R))

In [ ]:
np.set_printoptions(precision=8, suppress=True)

# R2s, t2s, M2s = utils_F._get_M2s(E_gt_th)
R2s, t2s, M2s = utils_F._get_M2s(torch.from_numpy(E_est))

R1, R2 = R2s[0].numpy(), R2s[1].numpy()
t1, t2 = t2s[0].numpy(), t2s[1].numpy()
t1 = t1 / np.linalg.norm(t1)
t2 = t2 / np.linalg.norm(t2)
print('R1:', R1, np.linalg.det(R1))
print('R2:', R2, np.linalg.det(R2))

R_gt = delta_Rtij[:3, :3]
t_gt = delta_Rtij[:3, 3:4]
t_gt = t_gt / np.linalg.norm(t_gt)
print('---R_gt: ', R_gt, np.linalg.det(R_gt))
# from scipy.linalg import qr
# Q, R = qr(R_gt)
# print (Q.dot(Q.T))

U, S, V = np.linalg.svd(R1)
R_test = U@V
print('---R_test: ', R_test, np.linalg.det(R_test))

from pyquaternion import Quaternion
q1 = Quaternion(matrix=R1)
print('===q1')
print(q1)
print(q1.degrees, q1.axis)
q2 = Quaternion(matrix=R2)
print('===q2')
print(q2)
print(q2.degrees, q1.axis)

# q = Quaternion(matrix=R2.T @ R1)
# print(q.degrees, q.axis)

print('===q_gt')
q_gt = Quaternion(matrix=R_gt)
print(q_gt, q_gt.axis)

def q_to_array(q):
    return np.array([q.real, q.imaginary[0], q.imaginary[1], q.imaginary[2]])

print('--L2 dists between (q1, q_gt), (q2, q_gt):')
n1 = np.linalg.norm(q_to_array(q_gt) - q_to_array(q1))
n2 = np.linalg.norm(q_to_array(q_gt) - q_to_array(q2))
print(n1, n2)

print('--L2 dists between (t1, t_gt), (t2, t_gt):')
n1 = np.linalg.norm(t_gt - t1)
n2 = np.linalg.norm(t_gt - t2)
print(n1, n2)

In [ ]:
print(q_to_array(q_gt), q_gt.axis, q_gt.degrees)
print(q_to_array(q1), q1.axis, q1.degrees)
print(q_to_array(q2), q2.axis, q2.degrees)

In [ ]:
print(t1.T)
print(t2.T)
print(t_gt.T)

In [ ]:
E, F = utils_F._E_F_from_Rt(torch.from_numpy(delta_Rtij_inv[:3, :3]), torch.from_numpy(delta_Rtij_inv[:3, 3:4]), K_th, tensor_input=True)
print(E.numpy().T)
print(E_gt)

## Triangulate points

In [ ]:
t_gt, R_gt

In [ ]:
t1, R1

In [ ]:
t2, R2

In [ ]:
M_estW

In [ ]:
# depths, est
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
# M2 = np.hstack((R_gt, t_gt))
M2 = np.hstack((R2, t2))
# M2 = M_estW
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]

import matplotlib.cm as cm
scores_show = X_tri[-1, unique_rows_all_idxes]
scores_show = np.clip(scores_show, 0., 80.)
plt.figure(figsize=(30, 2))
plt.hist(scores_show, 200)
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=scores_show, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.title('All corres by ours, colored with depth from triangulated EST pose')
plt.show()

# depths, gt
M1 = np.hstack((np.eye(3), np.zeros((3, 1))))
M2 = np.hstack((R_gt, t_gt))
# M2 = np.hstack((R1, t2))
# M2 = M_estW
X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]

import matplotlib.cm as cm
scores_show = X_tri[-1, unique_rows_all_idxes]
scores_show = np.clip(scores_show, 0., 80.)
plt.figure(figsize=(30, 2))
plt.hist(scores_show, 200)
plt.show()

plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], c=scores_show, edgecolors='w', linewidths=2., cmap=cm.plasma)
plt.colorbar()
plt.title('All corres by ours, colored with depth from triangulated GT pose')
plt.show()

# Test with imaginary pose

In [ ]:
M_estW

In [ ]:
E_test, F_test = utils_F.E_F_from_Rt_np(M_estW[:3, :3], delta_Rtij[:3, 3:4], sample['K'][0].numpy())
# E_test, F_test = utils_F.E_F_from_Rt_np(delta_Rtij[:3, :3], M_estW[:3, 3:4], sample['K'][0].numpy())

utils_vis.show_epipolar_rui_gtEst(x1[mask_conf, :], x2[mask_conf, :], img1_rgb, img2_rgb, F_gt, F_est, im_shape=im_shape, title_append='Ours top50 with largest score points')
utils_vis.show_epipolar_rui_gtEst(x1[mask_conf, :], x2[mask_conf, :], img1_rgb, img2_rgb, F_gt, F_test, im_shape=im_shape, title_append='Ours top50 with largest score points')


In [ ]:
sample['K'].numpy().shape

In [ ]:
delta_Rtij[:3, :3], delta_Rtij[:3, 3:4], sample['K']

### Scores

In [ ]:
# All our scores
scores_show = scores_ori[unique_rows_all_idxes]
scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=scores_show*50, c='r', edgecolors='w', linewidths=2.)
plt.title('All corres by ours')
plt.show()

# Scores of top ours
N = mask_sample.shape[0]
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
mask_conf = sort_idxes[:N]
scores_show = scores_ori[unique_rows_all_idxes][mask_conf]
scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], s=scores_show*50, c='r', edgecolors='w', linewidths=2.)
plt.title('Top %d(inlier num by OpenCV) by Ours'%N)
plt.show()

# Scores of top OpenCV
scores_show = scores_ori[mask_sample]
scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[mask_sample, 0], x1[mask_sample, 1], s=scores_show*50, c='r', edgecolors='w', linewidths=2.)
plt.title('Top %d by OpenCV'%N)
plt.show()

# Geo dists
geo_dists = np.sqrt(utils_F._sym_epi_dist(F_gt_th, torch.from_numpy(x1[unique_rows_all_idxes]), torch.from_numpy(x2[unique_rows_all_idxes])).numpy())
# plt.hist(geo_dists, 100)
# plt.show()
geo_dists = np.clip(geo_dists, 0, 10.)
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=geo_dists*50, c='r', edgecolors='w', linewidths=2.)
plt.show()

# All corres
utils_vis.draw_corr(img1_rgb, img2_rgb, x1[unique_rows_all_idxes], x2[unique_rows_all_idxes], linewidth=0., title='All corres')

### How much does out learned weights explain these corres (by showing the SVD res)

In [ ]:
from models.DeepFNet import NormalizeAndExpand_K
is_cuda = True
is_test = False


# norm_K = NormalizeAndExpand_K(is_cuda, is_test)
fit = Fit(is_cuda, is_test)

# pts = data_batch['matches_xy_ori']
# pts1, pts2, T1, T2 = norm_K(pts, data_batch['K_invs']) # pts: [b, N, 2] # \in [-1, 1]
# pts1 = pts1.permute(0,2,1)
# pts2 = pts2.permute(0,2,1)

out, X, p = fit(outs['pts1'], outs['pts2'], outs['weights'])
out = outs['T2'].permute(0,2,1).bmm(out.bmm(outs['T1'])).cpu().numpy()
X = X.cpu().numpy() # weighted
p = p.cpu().numpy() # un-weighted

In [ ]:
F_vec = out.reshape((9, 1))
print(F_vec.T)
# F_vec = F_gt.reshape((9, 1))
A = X.squeeze()
res = ((A @ F_vec)**2).flatten()

scores_show = res[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=scores_show*5000000000, c='r', edgecolors='w', linewidths=2.)
plt.title('All res by Ours (where we make errors in weighted SVD)')
plt.show()

scores_show = res[unique_rows_all_idxes][mask_conf]
# scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], s=scores_show*50000000000, c='r', edgecolors='w', linewidths=2.)
plt.title('Top %d(inlier num by OpenCV) res by Ours (where in out most conf. ones we make most errors)'%N)
plt.show()

# All our scores
scores_show = scores_ori[unique_rows_all_idxes]
scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=scores_show*50, c='r', edgecolors='w', linewidths=2.)
plt.title('Scores of all corres by ours')
plt.show()

# Scores of top ours
N = mask_sample.shape[0]
# N = 100
sort_idxes = np.argsort(scores_ori[unique_rows_all_idxes])[::-1]
mask_conf = sort_idxes[:N]
scores_show = scores_ori[unique_rows_all_idxes][mask_conf]
scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0][mask_conf], x1[unique_rows_all_idxes, 1][mask_conf], s=scores_show*50, c='r', edgecolors='w', linewidths=2.)
plt.title('Top %d scores(inlier num by OpenCV) by Ours'%N)
plt.show()

In [ ]:
# F_vec = out.reshape((9, 1))
F_vec = F_gt.reshape((9, 1))
print(F_vec.T)
A = p.squeeze()
res = ((A @ F_vec)**2).flatten()

scores_show = res[unique_rows_all_idxes]
# scores_show = scores_show / np.max(scores_show) * 2.
plt.figure(figsize=(30, 8))
plt.imshow(img1_rgb)
plt.scatter(x1[unique_rows_all_idxes, 0], x1[unique_rows_all_idxes, 1], s=scores_show*50000, c='r', edgecolors='w', linewidths=2.)
plt.title('All res by Ours')
plt.show()

### Filter points with largest res & Understanding the degenerate condition

In [ ]:
F_est

In [ ]:
from models.DeepFNet import NormalizeAndExpand_K
is_cuda = True
is_test = False

# norm_K = NormalizeAndExpand_K(is_cuda, is_test)
fit = Fit(is_cuda, is_test)

E_recover, X, p = fit(outs['pts1'], outs['pts2'], outs['weights'])
F_recover = outs['T2'].permute(0,2,1).bmm(E_recover.bmm(outs['T1']))
print(F_recover.cpu().numpy().squeeze())
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_recover.cpu().numpy().squeeze(), E_gt, F_recover.cpu().numpy().squeeze(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

# Get reses and filter out weights of largest reses
res = (X @ F_recover.view(-1, 9, 1))**2

res_rank = np.argsort(res.cpu().numpy().flatten())[::-1]
weights_np = outs['weights'].cpu().numpy() # [1, 1, 1000]
weights_np[:, :, res_rank[:10]] = 0.
# print(weights_np)
E_recover2, X, p = fit(outs['pts1'], outs['pts2'], torch.from_numpy(weights_np).cuda())
F_recover2 = outs['T2'].permute(0,2,1).bmm(E_recover2.bmm(outs['T1']))
print(F_recover2.cpu().numpy().squeeze())
error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_recover2.cpu().numpy().squeeze(), E_gt, F_recover2.cpu().numpy().squeeze(), F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

In [ ]:
print(delta_Rtij[:3])
utils_F._E_to_M(E_recover.squeeze().cpu(), K, x1, x2, inlier_mask=None, delta_Rt_gt=delta_Rtij_inv, show_debug=True)

In [ ]:
import operator
show_debug = True
show_result = True
E_est_th = E_recover.squeeze().cpu()
depth_thres = 100.
delta_Rt_gt = delta_Rtij_inv
method_name = 'Ours'

if show_debug:
    print('--- Recovering pose from E...')
count_N = x1.shape[0]
R2s, t2s, M2s = utils_F._get_M2s(E_est_th)

R1 = np.eye(3)
t1 = np.zeros((3, 1))
M1 = np.hstack((R1, t1))

# Cheirality check following OpenCV implementation: https://github.com/opencv/opencv/blob/808ba552c532408bddd5fe51784cf4209296448a/modules/calib3d/src/five-point.cpp#L513
depth_thres = depth_thres
cheirality_checks = []
M2_list = []
error_Rt = ()

def within_mask(Z, thres_min, thres_max):
    return (Z > thres_min) & (Z < thres_max)

X_tri_list = []
for Rt_idx, M2 in enumerate(M2s):
    M2 = M2.numpy()
    R2 = M2[:, :3]
    t2 = M2[:, 3:4]
    if show_debug:
        print(M2)
        # print(np.linalg.det(R2))
    X_tri_homo = cv2.triangulatePoints(np.matmul(K, M1), np.matmul(K, M2), x1.T, x2.T)
    X_tri = X_tri_homo[:3, :]/X_tri_homo[-1, :]
    X_tri_list.append(X_tri)
    # C1 = -np.matmul(R1, t1) # https://math.stackexchange.com/questions/82602/how-to-find-camera-position-and-rotation-from-a-4x4-matrix
    # cheirality1 = np.matmul(R1[2:3, :], (X_tri-C1)).reshape(-1) # https://cmsc426.github.io/sfm/
    # if show_debug:
    #     print(X_tri[-1, :])
    cheirality_mask_1 = within_mask(X_tri[-1, :], 0., depth_thres)

    X_tri_cam2 = np.matmul(R2, X_tri) + t2
    # C2 = -np.matmul(R2, t2)
    # cheirality2 = np.matmul(R2[2:3, :], (X_tri_cam3-C2)).reshape(-1)
    cheirality_mask_2 = within_mask(X_tri_cam2[-1, :], 0., depth_thres)

    cheirality_mask_12 = cheirality_mask_1 & cheirality_mask_2
    cheirality_checks.append(cheirality_mask_12)

    if show_debug:
        print('%d/%d points passed the check:'%(np.sum(cheirality_mask_12), X_tri.shape[1]))

if show_debug:
    print([np.sum(mask) for mask in cheirality_checks])

good_M_index, non_zero_nums = max(enumerate([np.sum(mask) for mask in cheirality_checks]), key=operator.itemgetter(1))
if non_zero_nums > 0:
    # Rt_idx = cheirality_checks.index(True)
    M_inv = utils_misc.Rt_depad(np.linalg.inv(utils_misc.Rt_pad(M2s[good_M_index].numpy())))
    if show_result:
        print('The %d_th (0-based) Rt meets the Cheirality Condition! with [R|t] (camera):\n'%good_M_index, M_inv)

    if delta_Rt_gt is not None:
        R2 = M2s[good_M_index][:, :3].numpy()
        t2 = M2s[good_M_index][:, 3:4].numpy()
        # error_R = min([utils_geo.rot12_to_angle_error(R2.numpy(), delta_R_gt) for R2 in R2s])
        # error_t = min(utils_geo.vector_angle(t2, delta_t_gt), utils_geo.vector_angle(-t2, delta_t_gt))

        R2 = M_inv[:, :3]
        t2 = M_inv[:, 3:4]
        error_R = utils_geo.rot12_to_angle_error(R2, delta_Rt_gt[:3, :3]) # [RUI] Both of camera motion
        error_t = utils_geo.vector_angle(t2, delta_Rt_gt[:3, 3:4])
        if show_result:
            print('Recovered by %s (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(method_name, error_R, error_t))
        error_Rt = (error_R, error_t)

else:
    # raise ValueError('ERROR! 0 of qualified [R|t] found!')
    print('ERROR! 0 of qualified [R|t] found!')
    error_Rt = ()


In [ ]:
scipy.io.savemat('Xs.mat', {'X_tri_list': X_tri_list})

### Mask center points

In [ ]:
# p1 = outs['pts1'].cpu().numpy().squeeze()[:, :2]
# p2 = outs['pts2'].cpu().numpy().squeeze()[:, :2]
# w = outs['weights'].cpu().numpy().squeeze()

# mask_x = np.logical_or(p1[:, 0]<-0.25, p1[:, 0]>0.25)
# mask_y = np.logical_or(p1[:, 1]<-0.1, p1[:, 0]>0.1)
# mask_xy = np.logical_and(mask_x, mask_y) 
mask_x = np.logical_and(p1[:, 0]>-0.25, p1[:, 0]<0.25)
mask_y = np.logical_and(p1[:, 1]>-0.1, p1[:, 1]<0.1)
mask_xy = np.logical_and(mask_x, mask_y) 

pts1_np = outs['pts1'].cpu().numpy()
pts2_np = outs['pts2'].cpu().numpy()
weights_np = outs['weights'].cpu().numpy()
# out_new, _, _ = fit(outs['pts1'], outs['pts2'], outs['weights'])
pts1_new = outs['pts1'].cpu().numpy()[:, np.logical_not(mask_xy), :]
pts2_new = outs['pts2'].cpu().numpy()[:, np.logical_not(mask_xy), :]
# weights_new = [:, :, mask_xy]
weights_np[:, :, mask_xy] = weights_np[:, :, mask_xy] * 0.

plt.subplot(121)
plt.scatter(p1[:, 0], p1[:, 1])
plt.subplot(122)
plt.scatter(pts1_new[0, :, 0], pts1_new[0, :, 1])
plt.show()

out_new, _, _ = fit(torch.from_numpy(pts1_np).cuda(), torch.from_numpy(pts2_np).cuda(), torch.from_numpy(weights_np).cuda())
F_ests_masked = outs['T2'].permute(0,2,1).bmm(out_new.bmm(outs['T1']))
F_est_masked = F_ests_masked.cpu().numpy().squeeze()
E_ests_masked = Ks.transpose(1, 2) @ F_ests_masked @ Ks
E_est_masked = E_ests_masked.cpu().detach().numpy()[0]


print(F_est_mask)
print(F_gt)
utils_vis.show_epipolar_rui_gtEst(x1[mask_xy, :][:30], x2[mask_xy, :][:30], img1_rgb, img2_rgb, F_gt, F_est_mask, im_shape=im_shape, title_append='Ours top50')

error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_est, E_gt, F_est, F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

error_Rt_est_ours, epi_dist_mean_est_ours, _, _, _, _, _, M_estW = val_rt(idx, K, x1, x2, E_est_masked, E_gt, F_est_mask, F_gt, delta_Rtij, five_point=False, if_opencv=False)
print('Recovered by ours (camera): The rotation error (degree) %.4f, and translation error (degree) %.4f'%(error_Rt_est_ours[0], error_Rt_est_ours[1]))

In [ ]:
class Fit(nn.Module):
    def __init__(self, is_cuda=True, is_test=False):
        super(Fit, self).__init__()
        # self.svd = bsvd(is_cuda, is_test)

        self.ones_b = Variable(torch.ones((1, 1, 1)).float())
        self.zero_b = Variable(torch.zeros((1, 1, 1)).float())
        self.T_b = torch.zeros(1, 3, 3).float()

        self.mask = Variable(torch.ones(3))
        self.mask[-1] = 0

        if is_cuda:
            self.ones_b = self.ones_b.cuda()
            self.zero_b = self.zero_b.cuda()
            self.T_b = self.T_b.cuda()
            self.mask = self.mask.cuda()
        self.is_cuda = is_cuda
#        self.bsvd = bsvd_torch()

    def normalize(self, pts, weights):
        device = pts.device
        T = Variable(self.T_b.to(device).expand(pts.size(0), 3, 3)).clone()
        ones = self.ones_b.to(device).expand(pts.size(0), pts.size(1), 1)

        denom = weights.sum(1)
        #

        # c = torch.mean(pts,1)
        # newpts_ = (pts - c.unsqueeze(1))
        # meandist = newpts_[:,:,:2].pow(2).sum(2).sqrt().mean(1)

        c = torch.sum(pts*weights,1)/denom
        # print(c.size(), pts.size())
        newpts_ = (pts - c.unsqueeze(1))
        meandist = ((weights*(newpts_[:,:,:2].pow(2).sum(2).sqrt().unsqueeze(2))).sum(1)/denom).squeeze(1)

        scale = 1.4142/meandist

        T[:,0,0] = scale
        T[:,1,1] = scale
        T[:,2,2] = 1
        T[:,0,2] = -c[:,0]*scale
        T[:,1,2] = -c[:,1]*scale

#        pts_ = torch.cat((pts, ones), 2)
        # print(pts.device, weights.device, T.device, self.T_b.device)
        pts_out = torch.bmm(T, pts.permute(0,2,1))
        return pts_out, T

    def weighted_svd(self, pts1, pts2, weights):
        device = weights.device
        weights = weights.squeeze(1).unsqueeze(2)

        ones = torch.ones_like(weights)
        if self.is_cuda:
            ones = ones.cuda()
        pts1n, T1 = self.normalize(pts1, ones)
        pts2n, T2 = self.normalize(pts2, ones)
        
        p = torch.cat((pts2n[:,0].unsqueeze(1)*pts1n,
                       pts2n[:,1].unsqueeze(1)*pts1n,
                       pts1n), 1).permute(0,2,1)

        p = torch.nn.functional.normalize(p, dim=2)

        X = p*weights

        out_b = []
        for b in range(X.size(0)):
            _, _, V = torch.svd(X[b])
            F = V[:,-1].view(3,3)
            U, S, V = torch.svd(F)
            F_ = U.mm((S*self.mask.to(device)).diag()).mm(V.t())
            out_b.append(F_.unsqueeze(0))

        out = torch.cat(out_b, 0)

        out = T2.permute(0,2,1).bmm(out).bmm(T1)

        return out, X, p

    def forward(self, pts1, pts2, weights):
        out, X, p = self.weighted_svd(pts1, pts2, weights)

        return out, X, p